In [ ]:
Objetivo: carregar somente os cabeçalhos (nomes das colunas) dos quatro arquivos CSV informados e mostrar em tela, de forma transposta, um quadro onde cada arquivo aparece como uma coluna e os nomes de campo ficam verticalmente.
Nada além disso será feito.

In [2]:
# ===============================================
# ETAPA: LEITURA DE CABEÇALHOS E EXIBIÇÃO TRANSPOSTA
# ===============================================

import os
import pandas as pd

# Arquivos informados (use exatamente estes caminhos)
csv_paths = [
    r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\pedra\A1_SECONDARIES_FOR_PEDRA_MODEL_old.csv",
    r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\pedra\A1_SECONDARIES_FOR_PEDRA_MODEL.csv",
    r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\pedra\A1_SECONDARIES_REFS.csv",
    r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\pedra\DELTA_PROXY_DIAGNOSTICS.csv",
]

def read_headers_only(path: str):
    """
    Lê apenas os cabeçalhos do CSV (sem carregar dados).
    Tenta primeiro utf-8-sig, depois latin1.
    Retorna lista de nomes de colunas.
    """
    # Verificação de existência
    if not os.path.exists(path):
        raise FileNotFoundError(f"Arquivo não encontrado: {path}")

    encodings = ["utf-8-sig", "latin1"]
    last_err = None
    for enc in encodings:
        try:
            # nrows=0 garante leitura apenas do header
            df = pd.read_csv(path, nrows=0, encoding=enc)
            return list(df.columns)
        except Exception as e:
            last_err = e
    # Se todas as tentativas falharem, reapresenta o último erro
    raise last_err

# Monta um DataFrame onde cada coluna representa um arquivo
series_by_file = {}
for p in csv_paths:
    headers = read_headers_only(p)
    name = os.path.basename(p)
    # Series indexadas por posição, para fácil visualização vertical
    series_by_file[name] = pd.Series(headers, dtype="object")

# Concatena lado a lado, alinhando pelo índice (posição dos cabeçalhos)
headers_matrix = pd.concat(series_by_file, axis=1)

# Exibição: nomes de arquivos como cabeçalhos de coluna
pd.set_option("display.max_rows", None)      # mostra todos os cabeçalhos
pd.set_option("display.max_columns", None)   # mostra todas as colunas (arquivos)
pd.set_option("display.width", 0)

print("Matriz transposta de cabeçalhos (cada coluna é um arquivo; linhas = nomes de campos na ordem original):")
display(headers_matrix)


Matriz transposta de cabeçalhos (cada coluna é um arquivo; linhas = nomes de campos na ordem original):


,A1_SECONDARIES_FOR_PEDRA_MODEL_old.csv,A1_SECONDARIES_FOR_PEDRA_MODEL.csv,A1_SECONDARIES_REFS.csv,DELTA_PROXY_DIAGNOSTICS.csv
0,Timestamp,timestamp,delta_proxy_ref,col
1,coal_flow_furnace_t_h,coal_flow_furnace_t_h,tau_densa_ref,group
2,total_air_flow_knm3_h,vazao_ar_total_knm3_h,tau_diluida_ref,nan_rate
3,total_paf_air_flow_knm3_h,vazao_ar_prim_total_knm3_h,tau_backpass_ref,spearman_tau
4,te_of_hot_pri_air_in_aph_outl_adegc,temp_hot_pri_air_in_preaq_ar_saida,v_proxy_total_ref,spearman_vtotal
5,tau_densa,tau_densa,v_proxy_primary_ref,spearman_vprimary
6,tau_diluida,tau_diluida,NaN,trend_tau
7,tau_backpass,tau_backpass,NaN,trend_vtotal
8,o2_excess_pct,o2_excess_pct,NaN,trend_vprimary
9,delta_proxy,delta_proxy,NaN,score


In [3]:
# ===============================================
# ETAPA: SALVAMENTO DO DATAFRAME DE CABEÇALHOS
# ===============================================

import os

# Caminho de saída
output_dir = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\analises_preliminares"
os.makedirs(output_dir, exist_ok=True)

# Arquivo final
output_path = os.path.join(output_dir, "matriz_cabecalhos.csv")

# Salvar DataFrame
headers_matrix.to_csv(output_path, index=True, encoding="utf-8-sig")

print(f"Arquivo salvo em: {output_path}")


Arquivo salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\analises_preliminares\matriz_cabecalhos.csv


In [4]:
# ===============================================
# ETAPA: VARREDURA DE CSVs (APENAS CABEÇALHOS)
# ===============================================
import os
import pandas as pd

# Diretório base do projeto
base_dir = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"

# Pastas a inspecionar (relativas a base_dir)
folders_to_scan = ["data", "outputs"]

# Colunas-alvo
target_cols = ["Wr", "Wm", "Wr_ref", "Wm_ref", "Wr_idx", "Wm_idx"]

# Onde salvar o inventário
output_dir = os.path.join(base_dir, "outputs")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "inventario_wr_wm.csv")

# Coletar resultados
results = []

for folder in folders_to_scan:
    folder_path = os.path.join(base_dir, folder)
    if not os.path.exists(folder_path):
        continue
    for root, _, files in os.walk(folder_path):
        for f in files:
            if not f.lower().endswith(".csv"):
                continue
            full_path = os.path.join(root, f)

            # Tenta ler só o header com duas codificações comuns
            df = None
            for enc in ("utf-8-sig", "latin1"):
                try:
                    df = pd.read_csv(full_path, nrows=0, encoding=enc)
                    break
                except Exception:
                    df = None

            if df is None:
                # Não conseguiu ler o cabeçalho — ignora
                continue

            found = [c for c in target_cols if c in df.columns]
            if found:
                results.append({
                    "arquivo": full_path,
                    "colunas_encontradas": ", ".join(found)
                })

# Monta DataFrame do inventário
df_inventory = pd.DataFrame(results, columns=["arquivo", "colunas_encontradas"])

# Salva inventário
df_inventory.to_csv(output_path, index=False, encoding="utf-8-sig")

# Exibe um resumo em tela (não lê dados, só informa)
print(f"Arquivos com colunas-alvo encontrados: {len(df_inventory)}")
print(f"Inventário salvo em: {output_path}")
if not df_inventory.empty:
    print("\nPrévia (até 10 linhas):")
    print(df_inventory.head(10).to_string(index=False))


Arquivos com colunas-alvo encontrados: 0
Inventário salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\inventario_wr_wm.csv


In [5]:
# ============================================================
# CONSOLIDAÇÃO MÍNIMA – AJUSTE DE CAMINHOS (MVP, SEM CÁLCULOS)
# ============================================================
import os
import pandas as pd

# Bases de caminho
SRC_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL"                    # leitura (já existente)
DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"         # escrita (criar se não existir)

# Garante que o diretório de destino exista
os.makedirs(DST_BASE, exist_ok=True)

# Fontes preferenciais (em A1_LOCAL)
SECUNDARIOS_CANDIDATOS = [
    os.path.join(SRC_BASE, "outputs", "pedra", "A1_SECONDARIES_FOR_PEDRA_MODEL.csv"),
    os.path.join(SRC_BASE, "outputs", "pedra", "A1_SECONDARIES_FOR_PEDRA_MODEL_old.csv"),
]
REFS_CANDIDATOS = [
    os.path.join(SRC_BASE, "outputs", "pedra", "A1_SECONDARIES_REFS.csv"),
]

# Saída padrão (em A1_LOCAL_REFAZIMENTO)
OUT_DIR  = os.path.join(DST_BASE, "data", "derivadas")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_PATH = os.path.join(OUT_DIR, "tabela_wr_wm_padrao.csv")

# Colunas alvo do padrão (sem cálculos; apenas preencher se existirem nas fontes)
PADRAO_COLS = [
    "timestamp","zona","componente",
    "flw_total_c_t_h",
    "total_air_flow_knm3_h","total_paf_air_flow_knm3_h",
    "te_of_hot_pri_air_in_aph_outl_adegc",
    "delta_proxy","tau_densa","tau_diluida","tau_backpass","o2_excess_pct",
    "Wr","Wm","Wr_ref","Wm_ref","Wr_idx","Wm_idx",
]

# Mapeamentos de nomes observados (não inferimos nada novo)
NAME_MAP = {
    "timestamp": "timestamp",
    "Timestamp": "timestamp",
    "vazao_ar_total_knm3_h": "total_air_flow_knm3_h",
    "vazao_ar_prim_total_knm3_h": "total_paf_air_flow_knm3_h",
    "temp_hot_pri_air_in_preaq_ar_saida": "te_of_hot_pri_air_in_aph_outl_adegc",
    "delta_proxy": "delta_proxy",
    "tau_densa": "tau_densa",
    "tau_diluida": "tau_diluida",
    "tau_backpass": "tau_backpass",
    "o2_excess_pct": "o2_excess_pct",
}

def read_csv_best_effort(path):
    for enc in ("utf-8-sig", "latin1"):
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except Exception:
            pass
    raise FileNotFoundError(f"Falha ao ler: {path}")

def pick_existing(paths):
    for p in paths:
        if os.path.exists(p):
            return p
    return None

# 1) Seleciona fontes existentes em A1_LOCAL
src_sec = pick_existing(SECUNDARIOS_CANDIDATOS)
src_ref = pick_existing(REFS_CANDIDATOS)

if src_sec is None:
    raise FileNotFoundError(
        "Não encontrei nenhum dos arquivos de secundárias esperados em A1_LOCAL\\outputs\\pedra\\ "
        "(A1_SECONDARIES_FOR_PEDRA_MODEL*.csv)."
    )

# 2) Carrega secundárias e aplica renomeações conhecidas
df_sec = read_csv_best_effort(src_sec)
rename_map = {c: NAME_MAP[c] for c in df_sec.columns if c in NAME_MAP}
df_sec = df_sec.rename(columns=rename_map)

if "timestamp" not in df_sec.columns:
    raise RuntimeError("A coluna 'timestamp' não foi encontrada após renomeação.")

# 3) Carrega referências (se existir), sem cálculos
df_ref = None
if src_ref is not None:
    try:
        df_ref = read_csv_best_effort(src_ref)
        ref_rename = {c: NAME_MAP[c] for c in df_ref.columns if c in NAME_MAP}
        if ref_rename:
            df_ref = df_ref.rename(columns=ref_rename)
    except Exception:
        df_ref = None

# 4) Expandir por 'zona' a partir de tau_* (apenas reshape; sem cálculos)
zonas_cols = [c for c in ["tau_densa","tau_diluida","tau_backpass"] if c in df_sec.columns]
if zonas_cols:
    id_cols = [c for c in df_sec.columns if c not in zonas_cols]
    df_long = df_sec.melt(
        id_vars=id_cols,
        value_vars=zonas_cols,
        var_name="zona",
        value_name="tau_val"
    )
else:
    df_long = df_sec.copy()
    df_long["zona"] = "[INFORMAÇÃO AUSENTE – PRECISAR PREENCHER]"

# 5) Componente ausente nas fontes → marcar como ausente (sem inferir)
df_long["componente"] = "[INFORMAÇÃO AUSENTE – PRECISAR PREENCHER]"

# 6) Merge leve com referências por timestamp (somente novas colunas)
if df_ref is not None and "timestamp" in df_ref.columns:
    cols_to_merge = [c for c in df_ref.columns if c != "timestamp" and c not in df_long.columns]
    if cols_to_merge:
        df_long = df_long.merge(df_ref[["timestamp"] + cols_to_merge], on="timestamp", how="left")

# 7) Garantir colunas do padrão (criar vazias quando ausentes)
for col in PADRAO_COLS:
    if col not in df_long.columns:
        df_long[col] = pd.NA

# 8) Reordenar e salvar em A1_LOCAL_REFAZIMENTO
df_final = df_long[PADRAO_COLS].copy()
df_final.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")

print(f"[OK] Tabela consolidada (sem cálculos) salva em:\n{OUT_PATH}")
print(f"Linhas: {len(df_final):,} | Colunas: {len(df_final.columns)}")
print("Colunas com algum preenchimento detectado:")
print([c for c in PADRAO_COLS if df_final[c].notna().any()])


[OK] Tabela consolidada (sem cálculos) salva em:
C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_padrao.csv
Linhas: 35,271 | Colunas: 18
Colunas com algum preenchimento detectado:
['timestamp', 'zona', 'componente', 'total_air_flow_knm3_h', 'total_paf_air_flow_knm3_h', 'te_of_hot_pri_air_in_aph_outl_adegc', 'delta_proxy', 'o2_excess_pct']


In [6]:
# ============================================================
# CONSOLIDAÇÃO (CORREÇÃO): PRESERVAR tau_* E REPLICAR POR ZONA
# ============================================================
import os
import pandas as pd

# Bases de caminho
SRC_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL"                    # leitura (já existente)
DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"         # escrita (criar se não existir)
os.makedirs(DST_BASE, exist_ok=True)

# Fontes em A1_LOCAL
SECUNDARIOS_CANDIDATOS = [
    os.path.join(SRC_BASE, "outputs", "pedra", "A1_SECONDARIES_FOR_PEDRA_MODEL.csv"),
    os.path.join(SRC_BASE, "outputs", "pedra", "A1_SECONDARIES_FOR_PEDRA_MODEL_old.csv"),
]
REFS_CANDIDATOS = [
    os.path.join(SRC_BASE, "outputs", "pedra", "A1_SECONDARIES_REFS.csv"),
]

# Saída em A1_LOCAL_REFAZIMENTO
OUT_DIR  = os.path.join(DST_BASE, "data", "derivadas")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_PATH = os.path.join(OUT_DIR, "tabela_wr_wm_padrao.csv")

# Padrão de colunas (sem cálculos)
PADRAO_COLS = [
    "timestamp","zona","componente",
    "flw_total_c_t_h",
    "total_air_flow_knm3_h","total_paf_air_flow_knm3_h",
    "te_of_hot_pri_air_in_aph_outl_adegc",
    "delta_proxy","tau_densa","tau_diluida","tau_backpass","o2_excess_pct",
    "Wr","Wm","Wr_ref","Wm_ref","Wr_idx","Wm_idx",
]

# Mapeamentos de nomes observados
NAME_MAP = {
    # timestamp
    "timestamp": "timestamp",
    "Timestamp": "timestamp",

    # vazões de ar
    "vazao_ar_total_knm3_h": "total_air_flow_knm3_h",
    "total_air_flow_knm3_h": "total_air_flow_knm3_h",
    "vazao_ar_prim_total_knm3_h": "total_paf_air_flow_knm3_h",
    "total_paf_air_flow_knm3_h": "total_paf_air_flow_knm3_h",

    # temperatura ar primário (APH)
    "temp_hot_pri_air_in_preaq_ar_saida": "te_of_hot_pri_air_in_aph_outl_adegc",
    "te_of_hot_pri_air_in_aph_outl_adegc": "te_of_hot_pri_air_in_aph_outl_adegc",

    # proxies / temps / O2
    "delta_proxy": "delta_proxy",
    "tau_densa": "tau_densa",
    "tau_diluida": "tau_diluida",
    "tau_backpass": "tau_backpass",
    "o2_excess_pct": "o2_excess_pct",

    # carvão
    "coal_flow_furnace_t_h": "flw_total_c_t_h",
    "flw_total_c_t_h": "flw_total_c_t_h",
}

def read_csv_best_effort(path):
    for enc in ("utf-8-sig", "latin1"):
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except Exception:
            pass
    raise FileNotFoundError(f"Falha ao ler: {path}")

def pick_existing(paths):
    for p in paths:
        if os.path.exists(p):
            return p
    return None

# 1) Selecionar fontes
src_sec = pick_existing(SECUNDARIOS_CANDIDATOS)
src_ref = pick_existing(REFS_CANDIDATOS)

if src_sec is None:
    raise FileNotFoundError(
        "Não encontrei arquivos de secundárias em A1_LOCAL\\outputs\\pedra\\ "
        "(A1_SECONDARIES_FOR_PEDRA_MODEL*.csv)."
    )

# 2) Carregar e renomear colunas conhecidas
df_sec = read_csv_best_effort(src_sec)
rename_map = {c: NAME_MAP[c] for c in df_sec.columns if c in NAME_MAP}
df_sec = df_sec.rename(columns=rename_map)

if "timestamp" not in df_sec.columns:
    raise RuntimeError("A coluna 'timestamp' não foi encontrada após renomeação.")

# 3) Carregar referências (merge leve por timestamp, sem cálculos)
df_ref = None
if src_ref is not None:
    try:
        df_ref = read_csv_best_effort(src_ref)
        ref_rename = {c: NAME_MAP[c] for c in df_ref.columns if c in NAME_MAP}
        if ref_rename:
            df_ref = df_ref.rename(columns=ref_rename)
    except Exception:
        df_ref = None

# 4) Replicar por zonas disponíveis, PRESERVANDO tau_* como colunas
zonas_presentes = []
if "tau_densa" in df_sec.columns:
    zonas_presentes.append("densa")
if "tau_diluida" in df_sec.columns:
    zonas_presentes.append("diluida")
if "tau_backpass" in df_sec.columns:
    zonas_presentes.append("backpass")

if zonas_presentes:
    partes = []
    for z in zonas_presentes:
        parte = df_sec.copy()
        parte["zona"] = z
        partes.append(parte)
    df_z = pd.concat(partes, axis=0, ignore_index=True)
else:
    df_z = df_sec.copy()
    df_z["zona"] = "[INFORMAÇÃO AUSENTE – PRECISAR PREENCHER]"

# 5) Componente ausente → marcador
df_z["componente"] = "[INFORMAÇÃO AUSENTE – PRECISAR PREENCHER]"

# 6) Merge leve com referências (adiciona colunas que não existirem ainda)
if df_ref is not None and "timestamp" in df_ref.columns:
    cols_to_merge = [c for c in df_ref.columns if c != "timestamp" and c not in df_z.columns]
    if cols_to_merge:
        df_z = df_z.merge(df_ref[["timestamp"] + cols_to_merge], on="timestamp", how="left")

# 7) Garantir todas as colunas do padrão (criando vazias quando necessário)
for col in PADRAO_COLS:
    if col not in df_z.columns:
        df_z[col] = pd.NA

# 8) Reordenar e salvar
df_final = df_z[PADRAO_COLS].copy()
df_final.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")

print(f"[OK] Tabela corrigida salva em:\n{OUT_PATH}")
print(f"Linhas: {len(df_final):,} | Colunas: {len(df_final.columns)}")
print("Colunas com algum preenchimento detectado:")
print([c for c in PADRAO_COLS if df_final[c].notna().any()])


[OK] Tabela corrigida salva em:
C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_padrao.csv
Linhas: 35,271 | Colunas: 18
Colunas com algum preenchimento detectado:
['timestamp', 'zona', 'componente', 'flw_total_c_t_h', 'total_air_flow_knm3_h', 'total_paf_air_flow_knm3_h', 'te_of_hot_pri_air_in_aph_outl_adegc', 'delta_proxy', 'tau_densa', 'tau_diluida', 'tau_backpass', 'o2_excess_pct']


In [7]:
# ============================================================
# EXPORTAR TABELA PADRÃO PARA PARQUET (SEM CÁLCULOS)
# ============================================================
import os
import pandas as pd

DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"
IN_CSV   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_padrao.csv")
OUT_DIR  = os.path.join(DST_BASE, r"data\derivadas")
OUT_PQ   = os.path.join(OUT_DIR, "tabela_wr_wm_padrao.parquet")

# Garantir diretório de saída
os.makedirs(OUT_DIR, exist_ok=True)

# Ler CSV (conteúdo já consolidado anteriormente)
df = pd.read_csv(IN_CSV, low_memory=False)

# Salvar em Parquet (engine pyarrow, se disponível)
df.to_parquet(OUT_PQ, index=False)

# Validação rápida
df_check = pd.read_parquet(OUT_PQ)
print("[OK] Parquet gerado:")
print(" caminho:", OUT_PQ)
print(" shape :", df_check.shape)
print(" colunas:", list(df_check.columns))


[OK] Parquet gerado:
 caminho: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_padrao.parquet
 shape : (35271, 18)
 colunas: ['timestamp', 'zona', 'componente', 'flw_total_c_t_h', 'total_air_flow_knm3_h', 'total_paf_air_flow_knm3_h', 'te_of_hot_pri_air_in_aph_outl_adegc', 'delta_proxy', 'tau_densa', 'tau_diluida', 'tau_backpass', 'o2_excess_pct', 'Wr', 'Wm', 'Wr_ref', 'Wm_ref', 'Wr_idx', 'Wm_idx']


In [8]:
# ============================================================
# DIAGNÓSTICO DE DUPLICIDADES NO PARQUET
# ============================================================
import os
import pandas as pd

DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"
PQ_PATH  = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_padrao.parquet")

OUT_DIR  = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_DUP_ALL = os.path.join(OUT_DIR, "duplicatas_exatas.csv")
OUT_DUP_KEY = os.path.join(OUT_DIR, "duplicatas_por_chave_timestamp_zona_componente.csv")

# 1) Ler parquet
df = pd.read_parquet(PQ_PATH)
total = len(df)
print(f"[INFO] shape: {df.shape} (linhas={total}, colunas={df.shape[1]})")

# 2) Duplicatas EXATAS (todas as colunas iguais)
mask_dups_all = df.duplicated(keep=False)  # marca todos os membros de cada grupo duplicado
qtd_linhas_em_grupos_duplicados = int(mask_dups_all.sum())
qtd_linhas_duplicadas_alem_da_primeira = int(df.duplicated().sum())
qtd_unicas = int(len(df.drop_duplicates()))

print("\n[EXATAS]")
print(f" - Linhas pertencentes a grupos duplicados (inclui primeira ocorrência): {qtd_linhas_em_grupos_duplicados}")
print(f" - Linhas duplicadas além da primeira (contagem de 'extras'):          {qtd_linhas_duplicadas_alem_da_primeira}")
print(f" - Linhas únicas após remover exatas:                                 {qtd_unicas}")

# Exporta amostra das duplicatas exatas (se existir)
if qtd_linhas_em_grupos_duplicados > 0:
    df_dups_all = df[mask_dups_all].copy()
    # Para reduzir tamanho do arquivo, limitamos a 50k linhas na exportação (ajuste se quiser)
    if len(df_dups_all) > 50000:
        df_dups_all = df_dups_all.head(50000)
    df_dups_all.to_csv(OUT_DUP_ALL, index=False, encoding="utf-8-sig")
    print(f" - Inventário (amostra) salvo em: {OUT_DUP_ALL}")
else:
    print(" - Nenhuma duplicata exata encontrada.")

# 3) Duplicatas por CHAVE ['timestamp','zona','componente']
key_cols = [c for c in ["timestamp","zona","componente"] if c in df.columns]
if len(key_cols) == 3:
    grp = df.groupby(key_cols, dropna=False).size().reset_index(name="count")
    dup_keys = grp[grp["count"] > 1].sort_values("count", ascending=False)
    qtd_chaves_duplicadas = int(len(dup_keys))
    total_linhas_em_chaves_duplicadas = int(dup_keys["count"].sum())

    print("\n[POR CHAVE: timestamp, zona, componente]")
    print(f" - Quantidade de chaves duplicadas: {qtd_chaves_duplicadas}")
    print(f" - Total de linhas cobertas por essas chaves: {total_linhas_em_chaves_duplicadas}")

    if qtd_chaves_duplicadas > 0:
        # Exportar todas as chaves duplicadas
        dup_keys.to_csv(OUT_DUP_KEY, index=False, encoding="utf-8-sig")
        print(f" - Inventário de chaves duplicadas salvo em: {OUT_DUP_KEY}")
else:
    print("\n[POR CHAVE]")
    print(" - A checagem por chave foi pulada porque faltam colunas em ['timestamp','zona','componente'].")

# 4) Contagem por zona (útil para entender multiplicação esperada)
if "zona" in df.columns:
    print("\n[CONTAGEM POR ZONA]")
    print(df["zona"].value_counts(dropna=False).to_string())


[INFO] shape: (35271, 18) (linhas=35271, colunas=18)

[EXATAS]
 - Linhas pertencentes a grupos duplicados (inclui primeira ocorrência): 0
 - Linhas duplicadas além da primeira (contagem de 'extras'):          0
 - Linhas únicas após remover exatas:                                 35271
 - Nenhuma duplicata exata encontrada.

[POR CHAVE: timestamp, zona, componente]
 - Quantidade de chaves duplicadas: 0
 - Total de linhas cobertas por essas chaves: 0

[CONTAGEM POR ZONA]
zona
densa       11757
diluida     11757
backpass    11757


In [9]:
# ============================================================
# COLAPSAR ZONAS -> 1 LINHA POR TIMESTAMP (SEM CÁLCULOS)
# ============================================================
import os
import pandas as pd

DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"
IN_PQ    = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_padrao.parquet")

OUT_DIR  = os.path.join(DST_BASE, r"data\derivadas")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_PQ   = os.path.join(OUT_DIR, "tabela_wr_wm_padrao_dedup.parquet")
OUT_CSV  = os.path.join(OUT_DIR, "tabela_wr_wm_padrao_dedup.csv")

DIAG_DIR = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(DIAG_DIR, exist_ok=True)
OUT_CONFLITOS = os.path.join(DIAG_DIR, "conflitos_por_timestamp.csv")

# 1) Ler parquet atual
df = pd.read_parquet(IN_PQ)
print("[INFO] shape original:", df.shape)

# 2) Verificar conflitos entre as 3 linhas por timestamp (excluindo 'zona')
cols_check = [c for c in df.columns if c != "zona"]
conf_linhas = []

# (Opcional) primeiro, confirmar a multiplicidade por timestamp
# (não bloqueia se não for sempre 3)
mult_por_ts = df.groupby("timestamp", dropna=False).size()

# Varredura de conflitos (se colunas variam dentro do mesmo timestamp)
for ts, grp in df.groupby("timestamp", dropna=False):
    diffs = [c for c in cols_check if grp[c].nunique(dropna=False) > 1]
    if diffs:
        conf_linhas.append({
            "timestamp": ts,
            "n_linhas_grupo": len(grp),
            "colunas_com_diferencas": ", ".join(diffs)
        })

# Exporta conflitos, se existirem
if conf_linhas:
    pd.DataFrame(conf_linhas).to_csv(OUT_CONFLITOS, index=False, encoding="utf-8-sig")
    print(f"[ATENÇÃO] Conflitos detectados por timestamp. Inventário salvo em:\n{OUT_CONFLITOS}")
else:
    print("[INFO] Nenhum conflito detectado entre as 3 linhas por timestamp (além de 'zona').")

# 3) Colapsar: remover 'zona' e deduplicar por 'timestamp' (mantendo a primeira ocorrência)
df_nz = df.drop(columns=["zona"], errors="ignore").copy()

# Ordena por timestamp para garantir determinismo na escolha da primeira
df_nz = df_nz.sort_values(by=["timestamp"]).copy()

# Remove duplicatas por timestamp
df_dedup = df_nz.drop_duplicates(subset=["timestamp"], keep="first").copy()

# 4) Salvar resultados
df_dedup.to_parquet(OUT_PQ, index=False)
df_dedup.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

print("\n[OK] Tabela deduplicada gerada:")
print(" - Parquet:", OUT_PQ)
print(" - CSV    :", OUT_CSV)
print("Shape final:", df_dedup.shape)

# 5) Validações auxiliares
if "timestamp" in df_dedup.columns:
    n_ts_unique = df_dedup["timestamp"].nunique(dropna=False)
    print("Total de timestamps únicos:", n_ts_unique)
else:
    print("[ALERTA] Coluna 'timestamp' ausente após deduplicação (não esperado).")


[INFO] shape original: (35271, 18)
[INFO] Nenhum conflito detectado entre as 3 linhas por timestamp (além de 'zona').

[OK] Tabela deduplicada gerada:
 - Parquet: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_padrao_dedup.parquet
 - CSV    : C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_padrao_dedup.csv
Shape final: (11757, 17)
Total de timestamps únicos: 11757


In [11]:
# ============================================================
# MVP DE MODELO: ISOLATION FOREST (CORREÇÃO NumPy 2.0)
# ============================================================
import os
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import Pipeline
import joblib

DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"

IN_PQ    = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_padrao_dedup.parquet")
DERIV_DIR = os.path.join(DST_BASE, r"data\derivadas")
MODEL_DIR = os.path.join(DST_BASE, r"outputs\modelos")
DIAG_DIR  = os.path.join(DST_BASE, r"outputs\diagnosticos")

os.makedirs(DERIV_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(DIAG_DIR, exist_ok=True)

# 1) Ler base deduplicada
df = pd.read_parquet(IN_PQ)

# 2) Seleção de features já preenchidas
features = [
    "flw_total_c_t_h",
    "total_air_flow_knm3_h",
    "total_paf_air_flow_knm3_h",
    "te_of_hot_pri_air_in_aph_outl_adegc",
    "delta_proxy",
    "tau_densa",
    "tau_diluida",
    "tau_backpass",
    "o2_excess_pct",
]
features = [c for c in features if c in df.columns]

# 3) Subconjunto de dados (timestamp + features)
df_model = df[["timestamp"] + features].copy()
for c in features:
    df_model[c] = pd.to_numeric(df_model[c], errors="coerce")

# 4) Pipeline: Imputer (mediana) + StandardScaler + IsolationForest
pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("iso", IsolationForest(
        n_estimators=300,
        max_samples="auto",
        contamination=0.03,   # ~3% anomalias
        random_state=42,
        n_jobs=-1
    ))
])

X = df_model[features].values
pipeline.fit(X)

# 5) Scores e rótulos (usando o pipeline para aplicar transformações)
# decision_function: maior = mais normal. Vamos inverter para anomalia (maior = mais anômalo).
raw_scores = pipeline.decision_function(X)  # array shape (n,)
anom_base = -raw_scores
rng = np.ptp(anom_base)  # max - min (NumPy 2.0+)
if not np.isfinite(rng) or rng == 0:
    rng = 1e-12
anomalia_score = (anom_base - np.min(anom_base)) / rng  # 0..1

pred_labels = pipeline.predict(X)  # -1 anomalia, 1 normal
is_anomalia = (pred_labels == -1).astype(int)

# 6) Montar saída
out = df_model.copy()
out["anomalia_score"] = anomalia_score
out["is_anomalia"] = is_anomalia

# 7) Salvar derivados
out_pq  = os.path.join(DERIV_DIR, "anomalias_isoforest.parquet")
out_csv = os.path.join(DERIV_DIR, "anomalias_isoforest.csv")
out.to_parquet(out_pq, index=False)
out.to_csv(out_csv, index=False, encoding="utf-8-sig")

# 8) Salvar modelo
model_path = os.path.join(MODEL_DIR, "isoforest_baseline.pkl")
joblib.dump(pipeline, model_path)

# 9) Estatísticas simples
pct_anom = 100.0 * out["is_anomalia"].mean()
sumario = []
sumario.append(f"Registros: {len(out):,}")
sumario.append(f"Features usadas: {features}")
sumario.append(f"% anomalias (contamination alvo=3%): {pct_anom:.2f}%")
top5 = out.sort_values("anomalia_score", ascending=False).head(5)[["timestamp","anomalia_score"]]
sumario.append("\nTop 5 timestamps mais anômalos (maior score):")
sumario.append(top5.to_string(index=False))

diag_path = os.path.join(DIAG_DIR, "anomalia_stats.txt")
with open(diag_path, "w", encoding="utf-8") as f:
    f.write("\n".join(sumario))

print("[OK] Derivados salvos:")
print(" -", out_pq)
print(" -", out_csv)
print("[OK] Modelo salvo em:", model_path)
print("\nResumo:")
print("\n".join(sumario))


[OK] Derivados salvos:
 - C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\anomalias_isoforest.parquet
 - C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\anomalias_isoforest.csv
[OK] Modelo salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\modelos\isoforest_baseline.pkl

Resumo:
Registros: 11,757
Features usadas: ['flw_total_c_t_h', 'total_air_flow_knm3_h', 'total_paf_air_flow_knm3_h', 'te_of_hot_pri_air_in_aph_outl_adegc', 'delta_proxy', 'tau_densa', 'tau_diluida', 'tau_backpass', 'o2_excess_pct']
% anomalias (contamination alvo=3%): 3.00%

Top 5 timestamps mais anômalos (maior score):
          timestamp  anomalia_score
2023-07-04 04:00:00        1.000000
2023-05-31 11:00:00        0.996667
2023-11-16 19:00:00        0.990161
2023-05-31 09:00:00        0.954710
2023-10-11 21:00:00        0.926770


In [12]:
# ============================================================
# PHYSICS-BASED (MVP): Wr/Wm + refs + índices
# ============================================================
import os
import json
import numpy as np
import pandas as pd

SRC_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL"
DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"

IN_PQ   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_padrao_dedup.parquet")
REFS_CSV= os.path.join(SRC_BASE, r"outputs\pedra\A1_SECONDARIES_REFS.csv")

OUT_DIR = os.path.join(DST_BASE, r"data\derivadas")
LOG_DIR = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

OUT_PQ  = os.path.join(OUT_DIR, "tabela_wr_wm_com_desgaste.parquet")
OUT_CSV = os.path.join(OUT_DIR, "tabela_wr_wm_com_desgaste.csv")
OUT_LOG = os.path.join(LOG_DIR, "physics_params_mvp.json")

# --- 1) Ler base deduplicada ---
df = pd.read_parquet(IN_PQ)

# Garantir numérico nas colunas usadas
def to_num(s):
    return pd.to_numeric(s, errors="coerce")

for c in ["delta_proxy","tau_densa","tau_diluida","tau_backpass",
          "total_air_flow_knm3_h","total_paf_air_flow_knm3_h","flw_total_c_t_h"]:
    if c in df.columns:
        df[c] = to_num(df[c])

# --- 2) Ler referências (se houver) ---
def read_csv_best_effort(path):
    for enc in ("utf-8-sig","latin1"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    return None

refs = read_csv_best_effort(REFS_CSV)

# Extrair refs como únicos valores (esperado 1 linha). Se houver várias, pegar a 1ª não nula.
def pick_ref(series):
    if series is None:
        return np.nan
    s = pd.to_numeric(series, errors="coerce")
    s = s.dropna()
    return s.iloc[0] if len(s) else np.nan

ref_vals = {}
if refs is not None:
    for col in ["delta_proxy_ref","tau_densa_ref","tau_diluida_ref","tau_backpass_ref",
                "v_proxy_total_ref","v_proxy_primary_ref"]:
        if col in refs.columns:
            ref_vals[col] = pick_ref(refs[col])
        else:
            ref_vals[col] = np.nan
else:
    # sem arquivo de refs
    ref_vals = {k: np.nan for k in
        ["delta_proxy_ref","tau_densa_ref","tau_diluida_ref","tau_backpass_ref","v_proxy_total_ref","v_proxy_primary_ref"]}

# --- 3) Definir variáveis de trabalho (MVP) ---
# τ (°C) -> τK (K). Usaremos média das três zonas disponíveis.
taus = [c for c in ["tau_densa","tau_diluida","tau_backpass"] if c in df.columns]
df["tau_mean_C"] = np.nan
if taus:
    df["tau_mean_C"] = pd.concat([df[t] for t in taus], axis=1).mean(axis=1, skipna=True)

# v (proxy de velocidade): preferimos ar total; se não tiver, ar primário; fallback: mediana.
if "total_air_flow_knm3_h" in df.columns:
    v_curr = df["total_air_flow_knm3_h"].copy()
elif "total_paf_air_flow_knm3_h" in df.columns:
    v_curr = df["total_paf_air_flow_knm3_h"].copy()
else:
    v_curr = pd.Series(np.nan, index=df.index)

# delta_proxy atual
delta_curr = df["delta_proxy"] if "delta_proxy" in df.columns else pd.Series(np.nan, index=df.index)

# Refs: se existirem no CSV, usamos; senão, fallback pela mediana dos próprios dados
v_ref = ref_vals.get("v_proxy_total_ref", np.nan)
if not np.isfinite(v_ref):
    v_ref = np.nanmedian(v_curr.values)

delta_ref = ref_vals.get("delta_proxy_ref", np.nan)
if not np.isfinite(delta_ref):
    delta_ref = np.nanmedian(delta_curr.values)

tau_ref_C_vals = [ref_vals.get("tau_densa_ref", np.nan),
                  ref_vals.get("tau_diluida_ref", np.nan),
                  ref_vals.get("tau_backpass_ref", np.nan)]
tau_ref_C = np.nanmean([x for x in tau_ref_C_vals if np.isfinite(x)]) if np.any(np.isfinite(tau_ref_C_vals)) else np.nan
if not np.isfinite(tau_ref_C):
    tau_ref_C = np.nanmedian(df["tau_mean_C"].values)

# --- 4) Constantes do MVP (ajustáveis) ---
params = {
    "alpha": 1.0,        # fatores escala (cancelam nos índices)
    "beta":  1.0,
    "n_r":   2.0,        # expoente de v para Wr
    "m_r":   1.0,        # expoente de delta para Wr
    "n_m":   3.0,        # expoente de v para Wm
    "m_m":   0.5,        # expoente de delta para Wm
    "Tcrit": 1200.0,     # K (refratário)
    "Topt":  1250.0      # K (metálico)
}

# --- 5) Cálculo (Wr/Wm e refs) ---
# Preparos
tauK     = to_num(df["tau_mean_C"]) + 273.15
tau_refK = float(tau_ref_C) + 273.15

v_curr_v = to_num(v_curr).values
delta_v  = to_num(delta_curr).values

# Wr e Wm atuais (proxy adimensional de desgaste)
Wr = params["alpha"] * np.power(v_curr_v, params["n_r"]) * np.power(delta_v, params["m_r"]) * np.exp(-tauK.values / params["Tcrit"])
Wm = params["beta"]  * np.power(v_curr_v, params["n_m"]) * np.power(delta_v, params["m_m"]) * np.exp(-tauK.values / params["Topt"])

# Wr_ref e Wm_ref (constantes, usando refs)
Wr_ref = params["alpha"] * (v_ref ** params["n_r"]) * (delta_ref ** params["m_r"]) * np.exp(-tau_refK / params["Tcrit"])
Wm_ref = params["beta"]  * (v_ref ** params["n_m"]) * (delta_ref ** params["m_m"]) * np.exp(-tau_refK / params["Topt"])

# Índices (cuidados para divisão por zero/NaN)
def safe_div(a, b):
    out = np.full_like(a, np.nan, dtype="float64")
    if b is not None and np.isfinite(b) and b != 0:
        out = a / b
    return out

Wr_idx = safe_div(Wr, Wr_ref)
Wm_idx = safe_div(Wm, Wm_ref)

# --- 6) Gravar na tabela final (não sobrescreve a original) ---
df_out = df.copy()
df_out["Wr"] = Wr
df_out["Wm"] = Wm
df_out["Wr_ref"] = Wr_ref if np.isfinite(Wr_ref) else np.nan
df_out["Wm_ref"] = Wm_ref if np.isfinite(Wm_ref) else np.nan
df_out["Wr_idx"] = Wr_idx
df_out["Wm_idx"] = Wm_idx

df_out.to_parquet(OUT_PQ, index=False)
df_out.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

# --- 7) Log de parâmetros e referências usadas ---
log = {
    "refs_arquivo_encontrado": os.path.exists(REFS_CSV),
    "refs_utilizadas": ref_vals,
    "fallbacks": {
        "v_ref_usou_median?"     : not np.isfinite(ref_vals.get("v_proxy_total_ref", np.nan)),
        "delta_ref_usou_median?" : not np.isfinite(ref_vals.get("delta_proxy_ref", np.nan)),
        "tau_ref_usou_median?"   : not np.any(np.isfinite(tau_ref_C_vals)),
    },
    "params": params,
    "coluna_v_atual": "total_air_flow_knm3_h" if "total_air_flow_knm3_h" in df.columns
                      else ("total_paf_air_flow_knm3_h" if "total_paf_air_flow_knm3_h" in df.columns else None),
    "linhas": len(df_out),
    "colunas": list(df_out.columns)
}
with open(OUT_LOG, "w", encoding="utf-8") as f:
    json.dump(log, f, ensure_ascii=False, indent=2)

# --- 8) Resumo em tela ---
print("[OK] Tabela com desgaste salva:")
print(" -", OUT_PQ)
print(" -", OUT_CSV)
print("\nResumo rápido:")
print("  Wr_ref:", Wr_ref, " | Wm_ref:", Wm_ref)
print("  % NaN Wr_idx:", np.mean(~np.isfinite(Wr_idx)) * 100, "% | % NaN Wm_idx:", np.mean(~np.isfinite(Wm_idx)) * 100, "%")
print("\nTop 5 maiores Wr_idx:")
print(pd.DataFrame({"timestamp": df_out["timestamp"], "Wr_idx": Wr_idx}).sort_values("Wr_idx", ascending=False).head(5).to_string(index=False))
print("\nTop 5 maiores Wm_idx:")
print(pd.DataFrame({"timestamp": df_out["timestamp"], "Wm_idx": Wm_idx}).sort_values("Wm_idx", ascending=False).head(5).to_string(index=False))
print("\n[INFO] Parâmetros e fallbacks registrados em:", OUT_LOG)


[OK] Tabela com desgaste salva:
 - C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_com_desgaste.parquet
 - C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_com_desgaste.csv

Resumo rápido:
  Wr_ref: 166056.27285204153  | Wm_ref: 248106045.49944544
  % NaN Wr_idx: 36.437866802755806 % | % NaN Wm_idx: 81.20268776048312 %

Top 5 maiores Wr_idx:
          timestamp    Wr_idx
2024-09-29 14:00:00 12.490335
2024-03-29 05:00:00 11.579226
2024-03-29 16:00:00 11.470146
2024-03-29 02:00:00 11.241728
2024-03-29 01:00:00 10.761733

Top 5 maiores Wm_idx:
          timestamp   Wm_idx
2024-09-29 14:00:00 3.428456
2024-03-29 05:00:00 3.291170
2024-03-29 16:00:00 3.263653
2024-03-29 02:00:00 3.181821
2024-03-29 01:00:00 3.140810

[INFO] Parâmetros e fallbacks registrados em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\physics_params_mvp.json


C:\Users\wilso\AppData\Local\Temp\ipykernel_26704\1751241561.py:124: RuntimeWarning: invalid value encountered in power
  Wm = params["beta"]  * np.power(v_curr_v, params["n_m"]) * np.power(delta_v, params["m_m"]) * np.exp(-tauK.values / params["Topt"])


In [13]:
# ============================================================
# GRÁFICOS TEMPORAIS DE Wr_idx / Wm_idx + MARCAÇÃO DE BASELINE
# ============================================================
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

SRC_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL"
DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"

IN_PQ   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste.parquet")
LOG_JSON= os.path.join(DST_BASE, r"outputs\diagnosticos\physics_params_mvp.json")

# possíveis fontes de período-baseline (opcionais)
CANDIDATES_BASELINE = [
    os.path.join(SRC_BASE, r"outputs\baseline_datasets\physics_baseline_proxies.csv"),
    os.path.join(SRC_BASE, r"outputs\baseline_mask.csv"),
]

OUT_DIR = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)

# 1) Ler tabela com desgaste
df = pd.read_parquet(IN_PQ)
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.sort_values("timestamp")

# 2) Carregar log para informar se refs vieram de medianas
fallback_info = {}
if os.path.exists(LOG_JSON):
    with open(LOG_JSON, "r", encoding="utf-8") as f:
        log_data = json.load(f)
    fallback_info = log_data.get("fallbacks", {})

# 3) Descobrir período de baseline (se existir)
baseline_range = None
baseline_source = None

# tenta arquivo de proxies de baseline (min..max do timestamp)
bp = CANDIDATES_BASELINE[0]
if os.path.exists(bp):
    try:
        bdf = pd.read_csv(bp, encoding="utf-8-sig")
    except Exception:
        bdf = pd.read_csv(bp, encoding="latin1")
    if "timestamp" in bdf.columns:
        bdf["timestamp"] = pd.to_datetime(bdf["timestamp"], errors="coerce")
        bdf = bdf.dropna(subset=["timestamp"])
        if not bdf.empty:
            baseline_range = (bdf["timestamp"].min(), bdf["timestamp"].max())
            baseline_source = os.path.relpath(bp, SRC_BASE)

# tenta máscara booleana
if baseline_range is None:
    bm = CANDIDATES_BASELINE[1]
    if os.path.exists(bm):
        try:
            mdf = pd.read_csv(bm, encoding="utf-8-sig")
        except Exception:
            mdf = pd.read_csv(bm, encoding="latin1")
        if {"timestamp","is_baseline"}.issubset(mdf.columns):
            mdf["timestamp"] = pd.to_datetime(mdf["timestamp"], errors="coerce")
            mdf = mdf[mdf["is_baseline"]==1].dropna(subset=["timestamp"])
            if not mdf.empty:
                baseline_range = (mdf["timestamp"].min(), mdf["timestamp"].max())
                baseline_source = os.path.relpath(bm, SRC_BASE)

# 4) Função helper para plotar série e sombrear baseline
def plot_idx(df, col_idx, out_png, title):
    s = df[["timestamp", col_idx]].dropna().copy()
    if s.empty:
        print(f"[INFO] Não há dados válidos para {col_idx}. Gráfico não gerado.")
        return

    plt.figure(figsize=(12,4))
    plt.plot(s["timestamp"], s[col_idx])
    plt.axhline(1.0, linestyle="--")  # linha do índice 1.0

    if baseline_range is not None:
        plt.axvspan(baseline_range[0], baseline_range[1], alpha=0.15)
        subtitle = f"Baseline sombreado (fonte: {baseline_source})"
    else:
        # informar na figura que não há período explícito
        # (refs podem ter vindo de constantes/medianas)
        fb_txt = []
        for k,v in fallback_info.items():
            if isinstance(v, bool):
                fb_txt.append(f"{k}={'sim' if v else 'não'}")
        subtitle = "Sem período de baseline localizado. " + (", ".join(fb_txt) if fb_txt else "")

    plt.title(f"{title}\n{subtitle}")
    plt.xlabel("timestamp")
    plt.ylabel(col_idx)
    plt.tight_layout()
    plt.savefig(out_png, dpi=120)
    plt.close()
    print("[OK] Gráfico salvo em:", out_png)

# 5) Gerar gráficos
plot_idx(df, "Wr_idx", os.path.join(OUT_DIR, "wr_idx_timeline.png"), "Wr_idx ao longo do tempo")
plot_idx(df, "Wm_idx", os.path.join(OUT_DIR, "wm_idx_timeline.png"), "Wm_idx ao longo do tempo")

# 6) Feedback em texto (terminal)
if baseline_range is not None:
    print(f"[INFO] Período de baseline usado para sombreamento: {baseline_range[0]} → {baseline_range[1]}")
else:
    print("[INFO] Não foi encontrado período explícito de baseline nos artefatos padrão.")


[OK] Gráfico salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_idx_timeline.png
[OK] Gráfico salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wm_idx_timeline.png
[INFO] Período de baseline usado para sombreamento: 2024-03-04 20:00:00 → 2024-03-12 15:00:00


In [14]:
# ============================================================
# PHYSICS v2 — Referência pelos mínimos absolutos + |Δ| e ε
# ============================================================
import os, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

SRC_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL"
DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"

IN_PQ   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_padrao_dedup.parquet")

OUT_DIR = os.path.join(DST_BASE, r"data\derivadas")
DIAG_DIR= os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(DIAG_DIR, exist_ok=True)

OUT_PQ  = os.path.join(OUT_DIR, "tabela_wr_wm_com_desgaste_v2.parquet")
OUT_CSV = os.path.join(OUT_DIR, "tabela_wr_wm_com_desgaste_v2.csv")
OUT_LOG = os.path.join(DIAG_DIR, "physics_params_mvp_v2.json")

# ---------- 1) Ler base e preparar variáveis primárias ----------
df = pd.read_parquet(IN_PQ).copy()
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")

# Variáveis primárias
v_col = "total_air_flow_knm3_h" if "total_air_flow_knm3_h" in df.columns else (
        "total_paf_air_flow_knm3_h" if "total_paf_air_flow_knm3_h" in df.columns else None)

if v_col is None or "delta_proxy" not in df.columns:
    raise RuntimeError("Faltam variáveis primárias: não encontrei vazão de ar ('total_air_flow_knm3_h' ou 'total_paf_air_flow_knm3_h') e/ou 'delta_proxy'.")

# numéricos
for c in [v_col, "delta_proxy", "tau_densa", "tau_diluida", "tau_backpass"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# temperatura média (°C) e Kelvin
taus = [c for c in ["tau_densa","tau_diluida","tau_backpass"] if c in df.columns]
df["tau_mean_C"] = pd.concat([df[t] for t in taus], axis=1).mean(axis=1, skipna=True)
df["tau_K"] = df["tau_mean_C"] + 273.15

# proxies ajustados (magnitude + epsilon)
eps = 1e-9
df["v_mag"]     = np.maximum(np.abs(df[v_col].values), eps)
df["delta_mag"] = np.maximum(np.abs(df["delta_proxy"].values), eps)

# ---------- 2) Parâmetros do modelo (iguais ao MVP, só trocamos Δ→|Δ|) ----------
params = dict(
    alpha=1.0, beta=1.0,
    n_r=2.0, m_r=1.0,     # Wr ∝ v^2 · Δ^1
    n_m=3.0, m_m=0.5,     # Wm ∝ v^3 · Δ^0.5
    Tcrit=1200.0, Topt=1250.0,
    eps=eps, v_col=v_col, use_delta="|delta_proxy|"
)

# ---------- 3) Wr* e Wm* (apenas onde tudo é finito) ----------
valid = np.isfinite(df["v_mag"]) & np.isfinite(df["delta_mag"]) & np.isfinite(df["tau_K"])
Wr_star = np.full(len(df), np.nan, dtype="float64")
Wm_star = np.full(len(df), np.nan, dtype="float64")

idx = np.where(valid)[0]
if len(idx) == 0:
    raise RuntimeError("Nenhuma linha válida para cálculo após aplicar |Δ| e τ_K.")

v  = df.loc[valid, "v_mag"].values
d  = df.loc[valid, "delta_mag"].values
tK = df.loc[valid, "tau_K"].values

Wr_star_valid = params["alpha"] * np.power(v, params["n_r"]) * np.power(d, params["m_r"]) * np.exp(-tK/params["Tcrit"])
Wm_star_valid = params["beta"]  * np.power(v, params["n_m"]) * np.power(d, params["m_m"]) * np.exp(-tK/params["Topt"])

Wr_star[idx] = Wr_star_valid
Wm_star[idx] = Wm_star_valid

# ---------- 4) Referências: mínimos absolutos por componente ----------
Wr_ref = np.nanmin(Wr_star)
Wm_ref = np.nanmin(Wm_star)

# Garantia de positividade
if not np.isfinite(Wr_ref) or Wr_ref <= 0 or not np.isfinite(Wm_ref) or Wm_ref <= 0:
    raise RuntimeError("Referências inválidas (Wr_ref/Wm_ref não finitas ou ≤ 0). Verifique dados.")

# ---------- 5) Timestamp representativo para marcar no gráfico ----------
# critério: ponto válido que minimiza a distância aos mínimos normalizados
cand = df.loc[valid, ["timestamp"]].copy()
cand["Wr_star"] = Wr_star_valid
cand["Wm_star"] = Wm_star_valid
cand["dist2"] = (cand["Wr_star"]/Wr_ref - 1.0)**2 + (cand["Wm_star"]/Wm_ref - 1.0)**2
cand = cand.sort_values(["dist2","timestamp"])
t_ref_repr = cand.iloc[0]["timestamp"]

# ---------- 6) Índices normalizados (>=1 por construção) ----------
Wr_idx = Wr_star / Wr_ref
Wm_idx = Wm_star / Wm_ref

df_out = df.copy()
df_out["Wr"] = Wr_star
df_out["Wm"] = Wm_star
df_out["Wr_ref"] = Wr_ref
df_out["Wm_ref"] = Wm_ref
df_out["Wr_idx"] = Wr_idx
df_out["Wm_idx"] = Wm_idx

# ---------- 7) Salvar saídas v2 ----------
df_out.to_parquet(OUT_PQ, index=False)
df_out.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

# ---------- 8) Gráficos com baseline marcado (linha/intervalo estreito) ----------
# janela para sombreamento: usa passo de tempo mediano
df_t = df_out.dropna(subset=["timestamp"]).sort_values("timestamp")
if len(df_t) >= 2:
    dt = df_t["timestamp"].diff().median()
    if not pd.isna(dt) and dt > pd.Timedelta(0):
        t0 = t_ref_repr - dt/2
        t1 = t_ref_repr + dt/2
    else:
        t0 = t1 = t_ref_repr
else:
    t0 = t1 = t_ref_repr

def plot_idx(series_col, title, png_name):
    s = df_out[["timestamp", series_col]].dropna().sort_values("timestamp")
    if s.empty:
        print(f"[INFO] Série vazia para {series_col}.")
        return
    plt.figure(figsize=(12,4))
    plt.plot(s["timestamp"], s[series_col])
    plt.axhline(1.0, linestyle="--")
    if t0 == t1:
        plt.axvline(t_ref_repr, linestyle=":")
        subtitle = f"Baseline = {t_ref_repr}"
    else:
        plt.axvspan(t0, t1, alpha=0.15)
        subtitle = f"Baseline ≈ {t_ref_repr} (faixa ~1 passo)"
    plt.title(f"{title}\n{subtitle}")
    plt.xlabel("timestamp"); plt.ylabel(series_col)
    plt.tight_layout()
    out_png = os.path.join(DIAG_DIR, png_name)
    plt.savefig(out_png, dpi=120); plt.close()
    print("[OK] Gráfico salvo em:", out_png)

plot_idx("Wr_idx", "Wr_idx ao longo do tempo (v2)", "wr_idx_timeline_v2.png")
plot_idx("Wm_idx", "Wm_idx ao longo do tempo (v2)", "wm_idx_timeline_v2.png")

# ---------- 9) Log ----------
log = {
    "params": params,
    "t_ref_repr": str(t_ref_repr),
    "Wr_ref": float(Wr_ref),
    "Wm_ref": float(Wm_ref),
    "percent_nan_Wr_idx": float(np.mean(~np.isfinite(Wr_idx)) * 100.0),
    "percent_nan_Wm_idx": float(np.mean(~np.isfinite(Wm_idx)) * 100.0),
    "valid_rows_used": int(valid.sum()),
    "total_rows": int(len(df)),
}
with open(OUT_LOG, "w", encoding="utf-8") as f:
    json.dump(log, f, ensure_ascii=False, indent=2)

print("\n[OK] v2 gerada:")
print(" -", OUT_PQ)
print(" -", OUT_CSV)
print("Baseline representativo:", t_ref_repr)
print("Wr_ref:", Wr_ref, "  Wm_ref:", Wm_ref)
print("% NaN Wr_idx:", log["percent_nan_Wr_idx"], " | % NaN Wm_idx:", log["percent_nan_Wm_idx"])


[OK] Gráfico salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_idx_timeline_v2.png
[OK] Gráfico salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wm_idx_timeline_v2.png

[OK] v2 gerada:
 - C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_com_desgaste_v2.parquet
 - C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_com_desgaste_v2.csv
Baseline representativo: 2023-12-02 10:00:00
Wr_ref: 147.92725653180094   Wm_ref: 7204476.667854408
% NaN Wr_idx: 36.437866802755806  | % NaN Wm_idx: 36.437866802755806


In [15]:
# ============================================================
# WR_IDX & WM_IDX JUNTOS + BASELINE (AZUL)  |  VERSÃO COM ZOOM
# ============================================================
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

SRC_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL"
DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"

IN_PQ   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste_v2.parquet")
OUT_DIR = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)

# 1) Carrega série v2
df = pd.read_parquet(IN_PQ)
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.sort_values("timestamp")

# 2) Descobrir baseline "anterior" nos artefatos antigos e escolher o de MAIOR duração
candidates = []
bp = os.path.join(SRC_BASE, r"outputs\baseline_datasets\physics_baseline_proxies.csv")
if os.path.exists(bp):
    try:
        bdf = pd.read_csv(bp, encoding="utf-8-sig")
    except Exception:
        bdf = pd.read_csv(bp, encoding="latin1")
    if "timestamp" in bdf.columns:
        bdf["timestamp"] = pd.to_datetime(bdf["timestamp"], errors="coerce")
        bdf = bdf.dropna(subset=["timestamp"])
        if not bdf.empty:
            candidates.append(("physics_baseline_proxies.csv", bdf["timestamp"].min(), bdf["timestamp"].max()))

bm = os.path.join(SRC_BASE, r"outputs\baseline_mask.csv")
if os.path.exists(bm):
    try:
        mdf = pd.read_csv(bm, encoding="utf-8-sig")
    except Exception:
        mdf = pd.read_csv(bm, encoding="latin1")
    if {"timestamp","is_baseline"}.issubset(mdf.columns):
        mdf["timestamp"] = pd.to_datetime(mdf["timestamp"], errors="coerce")
        mdf = mdf[mdf["is_baseline"]==1].dropna(subset=["timestamp"])
        if not mdf.empty:
            candidates.append(("baseline_mask.csv", mdf["timestamp"].min(), mdf["timestamp"].max()))

baseline_range = None
baseline_source = None
if candidates:
    # escolhe o com maior duração (proxy de "maior estabilidade encontrada")
    spans = [(src, t0, t1, (t1 - t0)) for (src, t0, t1) in candidates if pd.notna(t0) and pd.notna(t1)]
    if spans:
        spans.sort(key=lambda x: x[3], reverse=True)
        baseline_source, t0, t1, _ = spans[0]
        baseline_range = (t0, t1)

# 3) Função de plot geral
def plot_both(df, xcol, y1, y2, title, subtitle, path_png, xlim=None, shade=None):
    s = df[[xcol, y1, y2]].dropna(subset=[xcol]).copy()
    plt.figure(figsize=(14,4.5))
    plt.plot(s[xcol], s[y1], label=y1)
    plt.plot(s[xcol], s[y2], label=y2)
    plt.axhline(1.0, linestyle="--", linewidth=1)

    if shade is not None:
        t0, t1 = shade
        plt.axvspan(t0, t1, color="blue", alpha=0.12)

    if xlim is not None:
        plt.xlim(xlim)

    ttl = title
    if subtitle:
        ttl += f"\n{subtitle}"
    plt.title(ttl)
    plt.xlabel(xcol)
    plt.ylabel("índice (≥ 1 no v2)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(path_png, dpi=130)
    plt.close()
    print("[OK] Gráfico salvo:", path_png)

# 4) Gráfico 1 — todo o período
subtitle_all = ""
shade = None
if baseline_range is not None:
    subtitle_all = f"Baseline anterior sombreado (fonte: {baseline_source})"
    shade = baseline_range
else:
    subtitle_all = "Baseline anterior não localizado nos artefatos antigos."

png1 = os.path.join(OUT_DIR, "wr_wm_idx_together_v2.png")
plot_both(df, "timestamp", "Wr_idx", "Wm_idx",
          "Wr_idx e Wm_idx ao longo do tempo (v2)",
          subtitle_all, png1, shade=shade)

# 5) Gráfico 2 — zoom no intervalo de baseline (se existir)
if baseline_range is not None:
    t0, t1 = baseline_range
    # pequena margem de 2% do intervalo para visualização
    pad = (t1 - t0) * 0.02
    xlim = (t0 - pad, t1 + pad)
    png2 = os.path.join(OUT_DIR, "wr_wm_idx_together_v2_zoom_baseline.png")
    plot_both(df, "timestamp", "Wr_idx", "Wm_idx",
              "Wr_idx e Wm_idx — ZOOM no baseline (v2)",
              f"Período: {t0} → {t1}", png2, xlim=xlim, shade=baseline_range)
else:
    print("[INFO] Sem intervalo de baseline anterior — gráfico de zoom não gerado.")


[OK] Gráfico salvo: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_wm_idx_together_v2.png
[OK] Gráfico salvo: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_wm_idx_together_v2_zoom_baseline.png


In [16]:
# ============================================================
# SCATTER EM QUADRANTES: X=Wr (v2)  |  Y=Wm (v2)
# EIXOS CRUZANDO EM (Wr_ref_old, Wm_ref_old)  —  REF ANTERIOR
# ============================================================
import os, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"
SRC_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL"

# Arquivos
V2_PQ   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste_v2.parquet")
V1_PQ   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste.parquet")  # fallback
V1_LOG  = os.path.join(DST_BASE, r"outputs\diagnosticos\physics_params_mvp.json")      # principal
OUT_DIR = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_PNG = os.path.join(OUT_DIR, "wr_wm_quadrantes_ref_anterior.png")

# 1) Ler dados v2 (pontos a plotar)
df = pd.read_parquet(V2_PQ)
df["Wr"] = pd.to_numeric(df["Wr"], errors="coerce")
df["Wm"] = pd.to_numeric(df["Wm"], errors="coerce")
df_plot = df[["Wr","Wm"]].dropna().copy()

# 2) Buscar referências ANTERIORES (v1) — preferir LOG; senão, parquet v1
Wr_ref_old = np.nan
Wm_ref_old = np.nan

if os.path.exists(V1_LOG):
    try:
        with open(V1_LOG, "r", encoding="utf-8") as f:
            log1 = json.load(f)
        Wr_ref_old = float(log1.get("Wr_ref", np.nan))
        Wm_ref_old = float(log1.get("Wm_ref", np.nan))
    except Exception:
        pass

if not np.isfinite(Wr_ref_old) or not np.isfinite(Wm_ref_old):
    if os.path.exists(V1_PQ):
        try:
            df_v1 = pd.read_parquet(V1_PQ)
            if "Wr_ref" in df_v1.columns and "Wm_ref" in df_v1.columns:
                # pega primeiro valor não nulo de cada
                Wr_ref_old = pd.to_numeric(df_v1["Wr_ref"], errors="coerce").dropna().iloc[0]
                Wm_ref_old = pd.to_numeric(df_v1["Wm_ref"], errors="coerce").dropna().iloc[0]
        except Exception:
            pass

# Último fallback: abortar se não achou ref anterior
if not np.isfinite(Wr_ref_old) or not np.isfinite(Wm_ref_old):
    raise RuntimeError("Não foi possível localizar os valores de referência ANTERIORES (v1). Verifique o log JSON ou o parquet v1.")

# 3) Contagens por quadrante em relação à (Wr_ref_old, Wm_ref_old)
q1 = (df_plot["Wr"] >= Wr_ref_old) & (df_plot["Wm"] >= Wm_ref_old)  # alta/alta
q2 = (df_plot["Wr"] <  Wr_ref_old) & (df_plot["Wm"] >= Wm_ref_old)  # baixa/alta
q3 = (df_plot["Wr"] <  Wr_ref_old) & (df_plot["Wm"] <  Wm_ref_old)  # baixa/baixa
q4 = (df_plot["Wr"] >= Wr_ref_old) & (df_plot["Wm"] <  Wm_ref_old)  # alta/baixa

counts = {
    "Q1 (Wr↑, Wm↑)": int(q1.sum()),
    "Q2 (Wr↓, Wm↑)": int(q2.sum()),
    "Q3 (Wr↓, Wm↓)": int(q3.sum()),
    "Q4 (Wr↑, Wm↓)": int(q4.sum()),
}
total = len(df_plot)

# 4) Plot
plt.figure(figsize=(7.5,7))
plt.scatter(df_plot["Wr"], df_plot["Wm"], s=10, alpha=0.35)

# Linhas de referência (eixos do “quadro de quadrantes”)
plt.axvline(Wr_ref_old, linestyle="--", linewidth=1)
plt.axhline(Wm_ref_old, linestyle="--", linewidth=1)

plt.xlabel("Wr (v2)")
plt.ylabel("Wm (v2)")
plt.title("Wr (X) vs Wm (Y) — Eixos nos valores de referência ANTERIORES (v1)")

# Anotações dos quadrantes (percentuais)
def pct(n): 
    return f"{n} ({(100*n/total):.1f}%)"

xmin, xmax = df_plot["Wr"].min(), df_plot["Wr"].max()
ymin, ymax = df_plot["Wm"].min(), df_plot["Wm"].max()
xpad = (xmax - xmin) * 0.03
ypad = (ymax - ymin) * 0.03

plt.text(Wr_ref_old + xpad, Wm_ref_old + ypad,     f"Q1: {pct(counts['Q1 (Wr↑, Wm↑)'])}")
plt.text(xmin + xpad,        Wm_ref_old + ypad,     f"Q2: {pct(counts['Q2 (Wr↓, Wm↑)'])}")
plt.text(xmin + xpad,        ymin + ypad,           f"Q3: {pct(counts['Q3 (Wr↓, Wm↓)'])}")
plt.text(Wr_ref_old + xpad,  ymin + ypad,           f"Q4: {pct(counts['Q4 (Wr↑, Wm↓)'])}")

plt.tight_layout()
plt.savefig(OUT_PNG, dpi=140)
plt.close()

print("[OK] Gráfico de quadrantes salvo em:", OUT_PNG)
print("Contagens por quadrante:")
for k, v in counts.items():
    print(" -", k, ":", v, f"({v/total:.1%})")
print("Referências (anteriores): Wr_ref_old =", Wr_ref_old, "| Wm_ref_old =", Wm_ref_old)


[OK] Gráfico de quadrantes salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_wm_quadrantes_ref_anterior.png
Contagens por quadrante:
 - Q1 (Wr↑, Wm↑) : 5812 (77.8%)
 - Q2 (Wr↓, Wm↑) : 114 (1.5%)
 - Q3 (Wr↓, Wm↓) : 1438 (19.2%)
 - Q4 (Wr↑, Wm↓) : 109 (1.5%)
Referências (anteriores): Wr_ref_old = 166056.27285204153 | Wm_ref_old = 248106045.49944544


In [17]:
# ============================================================
# QUADRANTES COM REGIÃO "NORMAL" (RETÂNGULO AO REDOR DA REF)
# - X = Wr (v2), Y = Wm (v2)
# - Eixos cruzando em (Wr_ref_old, Wm_ref_old) = referência ANTERIOR (v1)
# - Retângulo de normalidade: ±(pct) em torno de cada referência
# - Reconta pontos FORA do retângulo por quadrante
# ============================================================
import os, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# -------------------- CONFIGURÁVEL --------------------
# janela percentual ao redor das referências (ex.: 0.10 = ±10%)
R_WR_PCT = 0.10
R_WM_PCT = 0.10

DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"
SRC_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL"

V2_PQ   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste_v2.parquet")
V1_PQ   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste.parquet")  # fallback
V1_LOG  = os.path.join(DST_BASE, r"outputs\diagnosticos\physics_params_mvp.json")      # principal

OUT_DIR = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_PNG = os.path.join(OUT_DIR, "wr_wm_quadrantes_ref_anterior_com_roi.png")
OUT_CSV = os.path.join(OUT_DIR, "wr_wm_quadrantes_outliers.csv")

# -------------------- 1) DADOS --------------------
df = pd.read_parquet(V2_PQ)
df["Wr"] = pd.to_numeric(df["Wr"], errors="coerce")
df["Wm"] = pd.to_numeric(df["Wm"], errors="coerce")
df_plot = df[["Wr","Wm"]].dropna().copy()

# referências ANTERIORES (v1)
Wr_ref_old = np.nan
Wm_ref_old = np.nan

if os.path.exists(V1_LOG):
    try:
        with open(V1_LOG, "r", encoding="utf-8") as f:
            log1 = json.load(f)
        Wr_ref_old = float(log1.get("Wr_ref", np.nan))
        Wm_ref_old = float(log1.get("Wm_ref", np.nan))
    except Exception:
        pass

if not np.isfinite(Wr_ref_old) or not np.isfinite(Wm_ref_old):
    if os.path.exists(V1_PQ):
        try:
            df_v1 = pd.read_parquet(V1_PQ)
            if "Wr_ref" in df_v1.columns and "Wm_ref" in df_v1.columns:
                Wr_ref_old = pd.to_numeric(df_v1["Wr_ref"], errors="coerce").dropna().iloc[0]
                Wm_ref_old = pd.to_numeric(df_v1["Wm_ref"], errors="coerce").dropna().iloc[0]
        except Exception:
            pass

if not np.isfinite(Wr_ref_old) or not np.isfinite(Wm_ref_old):
    raise RuntimeError("Não foi possível localizar Wr_ref_old / Wm_ref_old (v1).")

# -------------------- 2) REGIÃO NORMAL (ROI) --------------------
wr_min = Wr_ref_old * (1 - R_WR_PCT)
wr_max = Wr_ref_old * (1 + R_WR_PCT)
wm_min = Wm_ref_old * (1 - R_WM_PCT)
wm_max = Wm_ref_old * (1 + R_WM_PCT)

inside_roi = (df_plot["Wr"].between(wr_min, wr_max)) & (df_plot["Wm"].between(wm_min, wm_max))
outside_roi = ~inside_roi

# -------------------- 3) QUADRANTES (APENAS FORA DA ROI) --------------------
q1 = outside_roi & (df_plot["Wr"] >= Wr_ref_old) & (df_plot["Wm"] >= Wm_ref_old)  # alta/alta
q2 = outside_roi & (df_plot["Wr"] <  Wr_ref_old) & (df_plot["Wm"] >= Wm_ref_old)  # baixa/alta
q3 = outside_roi & (df_plot["Wr"] <  Wr_ref_old) & (df_plot["Wm"] <  Wm_ref_old)  # baixa/baixa
q4 = outside_roi & (df_plot["Wr"] >= Wr_ref_old) & (df_plot["Wm"] <  Wm_ref_old)  # alta/baixa

counts = {
    "ROI (normais)": int(inside_roi.sum()),
    "Q1 (Wr↑, Wm↑)": int(q1.sum()),
    "Q2 (Wr↓, Wm↑)": int(q2.sum()),
    "Q3 (Wr↓, Wm↓)": int(q3.sum()),
    "Q4 (Wr↑, Wm↓)": int(q4.sum()),
}
total = len(df_plot)

# salvar CSV dos pontos fora com rótulo de quadrante
outliers = df_plot.loc[outside_roi].copy()
quadrant = np.where(q1, "Q1",
             np.where(q2, "Q2",
             np.where(q3, "Q3",
             np.where(q4, "Q4", "NA"))))
outliers["quadrante"] = quadrant[outside_roi.values]
outliers.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

# -------------------- 4) PLOT --------------------
plt.figure(figsize=(8.2,8.2))

# pontos fora (azul claro) e dentro da ROI (verde)
plt.scatter(df_plot.loc[outside_roi, "Wr"], df_plot.loc[outside_roi, "Wm"], s=10, alpha=0.35, label="Fora da ROI")
plt.scatter(df_plot.loc[inside_roi,  "Wr"], df_plot.loc[inside_roi,  "Wm"], s=10, alpha=0.65, label="Dentro da ROI")

# eixos de referência (anteriores)
plt.axvline(Wr_ref_old, linestyle="--", linewidth=1)
plt.axhline(Wm_ref_old, linestyle="--", linewidth=1)

# retângulo da ROI
rect = Rectangle((wr_min, wm_min), wr_max - wr_min, wm_max - wm_min,
                 fill=False, linestyle="-", linewidth=1.5)
plt.gca().add_patch(rect)

plt.xlabel("Wr (v2)")
plt.ylabel("Wm (v2)")
plt.title("Wr (X) vs Wm (Y) — Eixos: ref ANTERIOR (v1)  |  ROI = retângulo em torno da ref")

# anotações com contagens (%)
def pct(n): return f"{n} ({(100*n/total):.1f}%)"
xmin, xmax = df_plot["Wr"].min(), df_plot["Wr"].max()
ymin, ymax = df_plot["Wm"].min(), df_plot["Wm"].max()
xpad = (xmax - xmin) * 0.02
ypad = (ymax - ymin) * 0.02

plt.text(Wr_ref_old + xpad, Wm_ref_old + ypad,     f"Q1: {pct(counts['Q1 (Wr↑, Wm↑)'])}")
plt.text(xmin + xpad,        Wm_ref_old + ypad,     f"Q2: {pct(counts['Q2 (Wr↓, Wm↑)'])}")
plt.text(xmin + xpad,        ymin + ypad,           f"Q3: {pct(counts['Q3 (Wr↓, Wm↓)'])}")
plt.text(Wr_ref_old + xpad,  ymin + ypad,           f"Q4: {pct(counts['Q4 (Wr↑, Wm↓)'])}")
plt.text(wr_min + xpad, wm_max + ypad, f"ROI ±{int(R_WR_PCT*100)}% x ±{int(R_WM_PCT*100)}%", fontsize=9)

plt.legend(loc="upper left")
plt.tight_layout()
plt.savefig(OUT_PNG, dpi=140)
plt.close()

print("[OK] Gráfico salvo:", OUT_PNG)
print("Referências (v1):  Wr_ref_old =", Wr_ref_old, " | Wm_ref_old =", Wm_ref_old)
print("ROI (±%): Wr =", R_WR_PCT*100, "%  |  Wm =", R_WM_PCT*100, "%")
print("Janela absoluta: Wr ∈ [", wr_min, ",", wr_max, "] ; Wm ∈ [", wm_min, ",", wm_max, "]")
print("\nContagens (pontos FORA por quadrante) e DENTRO da ROI:")
for k, v in counts.items():
    print(" -", k, ":", v, f"({v/total:.1%})")
print("\nCSV com pontos fora por quadrante:", OUT_CSV)


[OK] Gráfico salvo: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_wm_quadrantes_ref_anterior_com_roi.png
Referências (v1):  Wr_ref_old = 166056.27285204153  | Wm_ref_old = 248106045.49944544
ROI (±%): Wr = 10.0 %  |  Wm = 10.0 %
Janela absoluta: Wr ∈ [ 149450.64556683737 , 182661.90013724568 ] ; Wm ∈ [ 223295440.9495009 , 272916650.04939 ]

Contagens (pontos FORA por quadrante) e DENTRO da ROI:
 - ROI (normais) : 319 (4.3%)
 - Q1 (Wr↑, Wm↑) : 5717 (76.5%)
 - Q2 (Wr↓, Wm↑) : 55 (0.7%)
 - Q3 (Wr↓, Wm↓) : 1328 (17.8%)
 - Q4 (Wr↑, Wm↓) : 54 (0.7%)

CSV com pontos fora por quadrante: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_wm_quadrantes_outliers.csv


In [18]:
# ============================================================
# AJUSTE DOS RÓTULOS DE QUADRANTES (SEM SOBREPOSIÇÃO)
# - Usa posições relativas em cada quadrante (longe da interseção)
# - Mantém ROI e contagens como antes
# ============================================================
import os, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# ---------- Caminhos ----------
DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"
V2_PQ   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste_v2.parquet")
V1_PQ   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste.parquet")
V1_LOG  = os.path.join(DST_BASE, r"outputs\diagnosticos\physics_params_mvp.json")
OUT_DIR = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_PNG = os.path.join(OUT_DIR, "wr_wm_quadrantes_ref_anterior_com_roi_labels_ok.png")
OUT_CSV = os.path.join(OUT_DIR, "wr_wm_quadrantes_outliers.csv")

# ---------- Parâmetros ----------
R_WR_PCT = 0.10   # ROI ±10% em Wr
R_WM_PCT = 0.10   # ROI ±10% em Wm

# ---------- Dados ----------
df = pd.read_parquet(V2_PQ)
df["Wr"] = pd.to_numeric(df["Wr"], errors="coerce")
df["Wm"] = pd.to_numeric(df["Wm"], errors="coerce")
df_plot = df[["Wr","Wm"]].dropna().copy()

# Refs anteriores (v1)
Wr_ref_old = np.nan
Wm_ref_old = np.nan
if os.path.exists(V1_LOG):
    try:
        with open(V1_LOG, "r", encoding="utf-8") as f:
            log1 = json.load(f)
        Wr_ref_old = float(log1.get("Wr_ref", np.nan))
        Wm_ref_old = float(log1.get("Wm_ref", np.nan))
    except Exception:
        pass
if not np.isfinite(Wr_ref_old) or not np.isfinite(Wm_ref_old):
    if os.path.exists(V1_PQ):
        df_v1 = pd.read_parquet(V1_PQ)
        Wr_ref_old = pd.to_numeric(df_v1["Wr_ref"], errors="coerce").dropna().iloc[0]
        Wm_ref_old = pd.to_numeric(df_v1["Wm_ref"], errors="coerce").dropna().iloc[0]

# ---------- ROI ----------
wr_min = Wr_ref_old * (1 - R_WR_PCT)
wr_max = Wr_ref_old * (1 + R_WR_PCT)
wm_min = Wm_ref_old * (1 - R_WM_PCT)
wm_max = Wm_ref_old * (1 + R_WM_PCT)

inside_roi  = df_plot["Wr"].between(wr_min, wr_max) & df_plot["Wm"].between(wm_min, wm_max)
outside_roi = ~inside_roi

# Quadrantes (fora da ROI)
q1 = outside_roi & (df_plot["Wr"] >= Wr_ref_old) & (df_plot["Wm"] >= Wm_ref_old)
q2 = outside_roi & (df_plot["Wr"] <  Wr_ref_old) & (df_plot["Wm"] >= Wm_ref_old)
q3 = outside_roi & (df_plot["Wr"] <  Wr_ref_old) & (df_plot["Wm"] <  Wm_ref_old)
q4 = outside_roi & (df_plot["Wr"] >= Wr_ref_old) & (df_plot["Wm"] <  Wm_ref_old)

counts = {
    "ROI (normais)": int(inside_roi.sum()),
    "Q1 (Wr↑, Wm↑)": int(q1.sum()),
    "Q2 (Wr↓, Wm↑)": int(q2.sum()),
    "Q3 (Wr↓, Wm↓)": int(q3.sum()),
    "Q4 (Wr↑, Wm↓)": int(q4.sum()),
}
total = len(df_plot)

# Salva CSV dos pontos fora por quadrante
outliers = df_plot.loc[outside_roi].copy()
quadrant = np.where(q1, "Q1", np.where(q2, "Q2", np.where(q3, "Q3", np.where(q4, "Q4", "NA"))))
outliers["quadrante"] = quadrant[outside_roi.values]
outliers.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

# ---------- Plot ----------
plt.figure(figsize=(8.2,8.2))
plt.scatter(df_plot.loc[outside_roi,"Wr"], df_plot.loc[outside_roi,"Wm"], s=10, alpha=0.35, label="Fora da ROI")
plt.scatter(df_plot.loc[inside_roi, "Wr"], df_plot.loc[inside_roi, "Wm"], s=10, alpha=0.65, label="Dentro da ROI")

# Eixos de referência
plt.axvline(Wr_ref_old, linestyle="--", linewidth=1)
plt.axhline(Wm_ref_old, linestyle="--", linewidth=1)

# Retângulo ROI
rect = Rectangle((wr_min, wm_min), wr_max-wr_min, wm_max-wm_min, fill=False, linestyle="-", linewidth=1.5)
plt.gca().add_patch(rect)

plt.xlabel("Wr (v2)")
plt.ylabel("Wm (v2)")
plt.title("Wr (X) vs Wm (Y) — ROI e quadrantes com rótulos reposicionados")

# ------ Rótulos sem sobreposição ------
xmin, xmax = df_plot["Wr"].min(), df_plot["Wr"].max()
ymin, ymax = df_plot["Wm"].min(), df_plot["Wm"].max()

# posições relativas (entra 15%–35% em cada quadrante, longe da interseção/ROI)
x_q1 = Wr_ref_old + 0.18*(xmax - Wr_ref_old)
y_q1 = Wm_ref_old + 0.18*(ymax - Wm_ref_old)

x_q2 = Wr_ref_old - 0.30*(Wr_ref_old - xmin)
y_q2 = Wm_ref_old + 0.18*(ymax - Wm_ref_old)

x_q3 = Wr_ref_old - 0.30*(Wr_ref_old - xmin)
y_q3 = Wm_ref_old - 0.30*(Wm_ref_old - ymin)

x_q4 = Wr_ref_old + 0.18*(xmax - Wr_ref_old)
y_q4 = Wm_ref_old - 0.30*(Wm_ref_old - ymin)

box = dict(boxstyle="round,pad=0.2", fc="white", ec="gray", alpha=0.85)
fmt = lambda n: f"{n} ({(100*n/total):.1f}%)"

plt.text(x_q1, y_q1, f"Q1: {fmt(counts['Q1 (Wr↑, Wm↑)'])}", ha="left",  va="bottom", bbox=box)
plt.text(x_q2, y_q2, f"Q2: {fmt(counts['Q2 (Wr↓, Wm↑)'])}", ha="right", va="bottom", bbox=box)
plt.text(x_q3, y_q3, f"Q3: {fmt(counts['Q3 (Wr↓, Wm↓)'])}", ha="right", va="top",    bbox=box)
plt.text(x_q4, y_q4, f"Q4: {fmt(counts['Q4 (Wr↑, Wm↓)'])}", ha="left",  va="top",    bbox=box)

plt.legend(loc="upper left")
plt.tight_layout()
plt.savefig(OUT_PNG, dpi=150)
plt.close()

print("[OK] Gráfico salvo:", OUT_PNG)
print("Contagens:")
for k, v in counts.items():
    print(" -", k, ":", v, f"({v/total:.1%})")
print("CSV (fora da ROI):", OUT_CSV)


[OK] Gráfico salvo: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_wm_quadrantes_ref_anterior_com_roi_labels_ok.png
Contagens:
 - ROI (normais) : 319 (4.3%)
 - Q1 (Wr↑, Wm↑) : 5717 (76.5%)
 - Q2 (Wr↓, Wm↑) : 55 (0.7%)
 - Q3 (Wr↓, Wm↓) : 1328 (17.8%)
 - Q4 (Wr↑, Wm↓) : 54 (0.7%)
CSV (fora da ROI): C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_wm_quadrantes_outliers.csv


In [21]:
# ============================================================
# ROI ESTATÍSTICA + QUADRANTES + ATRIBUIÇÃO DE DRIVERS (Q1)
# - Retângulo ±k·σ (por eixo) e Elipse de Mahalanobis (χ²)
# - Contagem por quadrante (fora da ROI escolhida)
# - Decomposição de drivers (v, |Δ|, τ) para Q1
# ============================================================
import os, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Ellipse

# -------------------- CONFIG --------------------
K_SIGMA   = 1.5       # retângulo ±k·σ (ajuste: 1.0, 1.5, 2.0...)
CHI2_P    = 0.95      # elipse p-valor (0.90, 0.95, 0.99)
USE_ELLIPSE = True    # True = usa elipse p/ "normalidade"; False = usa retângulo k·σ

DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"
SRC_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL"

V2_PQ   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste_v2.parquet")
V1_PQ   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste.parquet")
V1_LOG  = os.path.join(DST_BASE, r"outputs\diagnosticos\physics_params_mvp.json")
BASELINE_PROXIES = os.path.join(SRC_BASE, r"outputs\baseline_datasets\physics_baseline_proxies.csv")
BASELINE_MASK    = os.path.join(SRC_BASE, r"outputs\baseline_mask.csv")

OUT_DIR = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_PNG = os.path.join(OUT_DIR, "wr_wm_roi_estatistica_quadrantes.png")
OUT_CSV = os.path.join(OUT_DIR, "wr_wm_outliers_por_quadrante.csv")
OUT_DRV = os.path.join(OUT_DIR, "drivers_q1.csv")

# -------------------- 1) CARREGAR DADOS --------------------
df = pd.read_parquet(V2_PQ)
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
for c in ["Wr","Wm","delta_proxy","total_air_flow_knm3_h","total_paf_air_flow_knm3_h","tau_densa","tau_diluida","tau_backpass"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Referências ANTERIORES (v1)
Wr_ref_old = np.nan; Wm_ref_old = np.nan
v_ref_old = np.nan; delta_ref_old = np.nan; tau_refC_old = np.nan
if os.path.exists(V1_LOG):
    with open(V1_LOG, "r", encoding="utf-8") as f:
        log1 = json.load(f)
    Wr_ref_old = float(log1.get("Wr_ref", np.nan))
    Wm_ref_old = float(log1.get("Wm_ref", np.nan))
    refs_u = log1.get("refs_utilizadas", {})
    # v_ref_old (preferir total; senão primary)
    v_ref_old = refs_u.get("v_proxy_total_ref", refs_u.get("v_proxy_primary_ref", np.nan))
    delta_ref_old = refs_u.get("delta_proxy_ref", np.nan)
    tau_refC_old = np.nanmean([refs_u.get("tau_densa_ref", np.nan),
                               refs_u.get("tau_diluida_ref", np.nan),
                               refs_u.get("tau_backpass_ref", np.nan)])
# fallback às colunas do parquet v1 (se necessário)
if (not np.isfinite(Wr_ref_old)) or (not np.isfinite(Wm_ref_old)):
    if os.path.exists(V1_PQ):
        df1 = pd.read_parquet(V1_PQ)
        Wr_ref_old = pd.to_numeric(df1["Wr_ref"], errors="coerce").dropna().iloc[0]
        Wm_ref_old = pd.to_numeric(df1["Wm_ref"], errors="coerce").dropna().iloc[0]

# Checagem
if not (np.isfinite(Wr_ref_old) and np.isfinite(Wm_ref_old)):
    raise RuntimeError("Não foi possível recuperar Wr_ref_old/Wm_ref_old.")

# Wr/Wm válidos p/ scatter
XY = df[["Wr","Wm","timestamp"]].dropna().copy()

# -------------------- 2) DEFINIR BASELINE PARA σ e Σ --------------------
baseline_range = None
if os.path.exists(BASELINE_PROXIES):
    bdf = pd.read_csv(BASELINE_PROXIES, encoding="utf-8-sig")
    if "timestamp" in bdf.columns:
        bdf["timestamp"] = pd.to_datetime(bdf["timestamp"], errors="coerce")
        bdf = bdf.dropna(subset=["timestamp"])
        if not bdf.empty:
            baseline_range = (bdf["timestamp"].min(), bdf["timestamp"].max())

if (baseline_range is None) and os.path.exists(BASELINE_MASK):
    mdf = pd.read_csv(BASELINE_MASK, encoding="utf-8-sig")
    if {"timestamp","is_baseline"}.issubset(mdf.columns):
        mdf["timestamp"] = pd.to_datetime(mdf["timestamp"], errors="coerce")
        mdf = mdf[mdf["is_baseline"]==1].dropna(subset=["timestamp"])
        if not mdf.empty:
            baseline_range = (mdf["timestamp"].min(), mdf["timestamp"].max())

if baseline_range is not None:
    t0, t1 = baseline_range
    base_df = df[(df["timestamp"]>=t0) & (df["timestamp"]<=t1)][["Wr","Wm"]].dropna().copy()
else:
    base_df = df[["Wr","Wm"]].dropna().copy()

if len(base_df) < 10:
    # se baseline muito pequeno, usa conjunto todo
    base_df = df[["Wr","Wm"]].dropna().copy()

# Estatísticas no baseline
mu_wr = base_df["Wr"].mean()
mu_wm = base_df["Wm"].mean()
sigma_wr = base_df["Wr"].std(ddof=1)
sigma_wm = base_df["Wm"].std(ddof=1)
cov = np.cov(base_df["Wr"], base_df["Wm"])

# -------------------- 3) ROI: retângulo ±k·σ OU elipse χ² --------------------
# Centro SEMPRE na ref antiga (pedido do usuário)
cx, cy = Wr_ref_old, Wm_ref_old

def inside_rectangle(x, y):
    return (x >= cx - K_SIGMA*sigma_wr) & (x <= cx + K_SIGMA*sigma_wr) & \
           (y >= cy - K_SIGMA*sigma_wm) & (y <= cy + K_SIGMA*sigma_wm)

# Qui-quadrado crítico (2 g.l.)
CHI2_TABLE = {0.90:4.605, 0.95:5.991, 0.99:9.210}
chi2_thr = CHI2_TABLE.get(CHI2_P, 5.991)

def inside_ellipse(x, y):
    # distância de Mahalanobis ao centro usando Σ do baseline
    diff = np.vstack([x - cx, y - cy])          # shape (2, n)
    try:
        inv_cov = np.linalg.inv(cov)
    except np.linalg.LinAlgError:
        # se cov singular, usa diagonal variâncias
        inv_cov = np.linalg.inv(np.diag([sigma_wr**2, sigma_wm**2]))
    d2 = np.einsum("ij,jk,ik->i", diff.T, inv_cov, diff.T)
    return d2 <= chi2_thr

# máscara de pontos "normais"
if USE_ELLIPSE:
    inside = inside_ellipse(XY["Wr"].values, XY["Wm"].values)
    roi_label = f"Elipse χ² (p={int(CHI2_P*100)}%)"
else:
    inside = inside_rectangle(XY["Wr"].values, XY["Wm"].values)
    roi_label = f"Retângulo ±{K_SIGMA}σ"

outside = ~inside

# -------------------- 4) QUADRANTES (fora da ROI) --------------------
x = XY["Wr"].values; y = XY["Wm"].values
q1 = outside & (x >= cx) & (y >= cy)
q2 = outside & (x <  cx) & (y >= cy)
q3 = outside & (x <  cx) & (y <  cy)
q4 = outside & (x >= cx) & (y <  cy)

counts = {
    f"ROI {roi_label} (normais)": int(inside.sum()),
    "Q1 (Wr↑, Wm↑)": int(q1.sum()),
    "Q2 (Wr↓, Wm↑)": int(q2.sum()),
    "Q3 (Wr↓, Wm↓)": int(q3.sum()),
    "Q4 (Wr↑, Wm↓)": int(q4.sum()),
}
total = len(XY)

# Salva CSV dos fora da ROI com quadrante
outliers = XY.loc[outside].copy()
quadrant = np.where(q1, "Q1", np.where(q2, "Q2", np.where(q3, "Q3", np.where(q4, "Q4","NA"))))
outliers["quadrante"] = quadrant[outside]
outliers.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

# -------------------- 5) ATRIBUIÇÃO DE DRIVERS (apenas Q1) --------------------
# Usamos a forma log-linear do MVP:
# Δlog Wr = 2·Δlog v + 1·Δlog |Δ| - Δτ/Tcrit
# Δlog Wm = 3·Δlog v + 0.5·Δlog |Δ| - Δτ/Topt
# Referências (v1) para comparar: v_ref_old, delta_ref_old, tau_refC_old
# Observação: usamos |Δ| (como no v2) e τK = τC + 273.15

# Preparar variáveis primárias
v_col = "total_air_flow_knm3_h" if "total_air_flow_knm3_h" in df.columns else "total_paf_air_flow_knm3_h"
v = pd.to_numeric(df[v_col], errors="coerce")
delta_mag = np.maximum(np.abs(pd.to_numeric(df["delta_proxy"], errors="coerce")), 1e-9)
tauC = pd.concat([df[c] for c in ["tau_densa","tau_diluida","tau_backpass"] if c in df.columns], axis=1).mean(axis=1, skipna=True)
tauK = tauC + 273.15

# refs antigas (módulo em Δ e Kelvin em τ)
if not np.isfinite(v_ref_old):
    v_ref_old = np.nanmedian(v)
if not np.isfinite(delta_ref_old):
    delta_ref_old = np.nanmedian(delta_mag)
tau_refK_old = (tau_refC_old if np.isfinite(tau_refC_old) else np.nanmedian(tauC)) + 273.15

# Log-diferenças e termos
dlogv = np.log(v / v_ref_old)
dlogd = np.log(delta_mag / np.abs(delta_ref_old))
dtaur = -(tauK - tau_refK_old) / 1200.0
dtaum = -(tauK - tau_refK_old) / 1250.0

dlogWr = 2.0*dlogv + 1.0*dlogd + dtaur
dlogWm = 3.0*dlogv + 0.5*dlogd + dtaum

# Seleciona as mesmas linhas do scatter OUTSIDE & Q1
mask_q1_idx = XY.index[outside & (x >= cx) & (y >= cy)]
drv = pd.DataFrame({
    "timestamp": df.loc[mask_q1_idx, "timestamp"],
    "dlogv": dlogv.loc[mask_q1_idx].values,
    "dlogd": dlogd.loc[mask_q1_idx].values,
    "dtaur": dtaur.loc[mask_q1_idx].values,
    "dtaum": dtaum.loc[mask_q1_idx].values,
    "dlogWr": dlogWr.loc[mask_q1_idx].values,
    "dlogWm": dlogWm.loc[mask_q1_idx].values,
})
# pesos médios absolutos por driver (combinando Wr e Wm)
w_v = 2.5 * drv["dlogv"].abs()                       # (2 + 3)/2
w_d = 0.75 * drv["dlogd"].abs()                      # (1 + 0.5)/2
w_t = 0.5 * (drv["dtaur"].abs() + drv["dtaum"].abs())

drv["w_v"] = w_v
drv["w_d"] = w_d
drv["w_t"] = w_t

# máscaras de dominância (booleans); .mean() em Series boolean retorna fração True
mask_v_domina = (w_v > w_d) & (w_v > w_t)
mask_d_domina = (w_d > w_v) & (w_d > w_t)
mask_t_domina = (w_t > w_v) & (w_t > w_d)

summary = pd.Series({
    "Q1_n": int(len(drv)),
    "peso_medio_v": w_v.mean(),
    "peso_medio_|Delta|": w_d.mean(),
    "peso_medio_temp": w_t.mean(),
    "%Q1_onde_v_domina": mask_v_domina.mean() * 100.0,
    "%Q1_onde_|Delta|_domina": mask_d_domina.mean() * 100.0,
    "%Q1_onde_temp_domina": mask_t_domina.mean() * 100.0
})

# (opcional) sobrescreve arquivo com as novas colunas de pesos
drv.to_csv(OUT_DRV, index=False, encoding="utf-8-sig")

print(summary.to_string())

# -------------------- 6) PLOT (elipse/retângulo + quadrantes) --------------------
plt.figure(figsize=(8.4,8.4))
plt.scatter(XY.loc[outside, "Wr"], XY.loc[outside, "Wm"], s=10, alpha=0.35, label="Fora da ROI")
plt.scatter(XY.loc[inside,  "Wr"], XY.loc[inside,  "Wm"], s=10, alpha=0.65, label="Dentro da ROI")
plt.axvline(cx, linestyle="--", linewidth=1)
plt.axhline(cy, linestyle="--", linewidth=1)

ax = plt.gca()
if USE_ELLIPSE:
    # desenha elipse aproximando eixos pela decomposição de cov
    vals, vecs = np.linalg.eigh(cov)
    vals = np.clip(vals, 1e-12, None)
    # fator para raio na elipse: sqrt(χ²)
    scale = np.sqrt(chi2_thr)
    width  = 2*scale*np.sqrt(vals[0])
    height = 2*scale*np.sqrt(vals[1])
    angle  = np.degrees(np.arctan2(vecs[1,0], vecs[0,0]))
    e = Ellipse((cx, cy), width, height, angle=angle, fill=False, linewidth=1.5)
    ax.add_patch(e)
    roi_txt = roi_label
else:
    rect = Rectangle((cx - K_SIGMA*sigma_wr, cy - K_SIGMA*sigma_wm),
                     2*K_SIGMA*sigma_wr, 2*K_SIGMA*sigma_wm,
                     fill=False, linewidth=1.5)
    ax.add_patch(rect)
    roi_txt = roi_label

plt.xlabel("Wr (v2)"); plt.ylabel("Wm (v2)")
plt.title(f"ROI estatística ({roi_txt}) e quadrantes")
# rótulos de quadrante afastados
xmin, xmax = XY["Wr"].min(), XY["Wr"].max()
ymin, ymax = XY["Wm"].min(), XY["Wm"].max()
xpadR = 0.18*(xmax - cx); xpadL = 0.30*(cx - xmin)
ypadU = 0.18*(ymax - cy); ypadD = 0.30*(cy - ymin)
box = dict(boxstyle="round,pad=0.2", fc="white", ec="gray", alpha=0.85)
fmt = lambda n: f"{n} ({(100*n/total):.1f}%)"
plt.text(cx + xpadR, cy + ypadU, f"Q1: {fmt(counts['Q1 (Wr↑, Wm↑)'])}", ha="left",  va="bottom", bbox=box)
plt.text(cx - xpadL, cy + ypadU, f"Q2: {fmt(counts['Q2 (Wr↓, Wm↑)'])}", ha="right", va="bottom", bbox=box)
plt.text(cx - xpadL, cy - ypadD, f"Q3: {fmt(counts['Q3 (Wr↓, Wm↓)'])}", ha="right", va="top",    bbox=box)
plt.text(cx + xpadR, cy - ypadD, f"Q4: {fmt(counts['Q4 (Wr↑, Wm↓)'])}", ha="left",  va="top",    bbox=box)

plt.legend(loc="upper left")
plt.tight_layout(); plt.savefig(OUT_PNG, dpi=150); plt.close()

# -------------------- 7) PRINTS --------------------
print("[OK] Gráfico salvo:", OUT_PNG)
print("Contagens (fora da ROI):")
for k, v in counts.items():
    print(" -", k, ":", v, f"({v/total:.1%})")
print("\nAtribuição de drivers em Q1 (arquivo):", OUT_DRV)
print(summary.to_string())
print("\nROI usada:", roi_txt)
if baseline_range is not None:
    print("Estatísticas estimadas no baseline:", baseline_range[0], "→", baseline_range[1])
else:
    print("Sem baseline explícito — σ e Σ estimados no conjunto todo (fallback).")


Q1_n                       3175.000000
peso_medio_v                  0.077052
peso_medio_|Delta|            0.905529
peso_medio_temp               0.000613
%Q1_onde_v_domina             1.763780
%Q1_onde_|Delta|_domina      98.236220
%Q1_onde_temp_domina          0.000000
[OK] Gráfico salvo: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_wm_roi_estatistica_quadrantes.png
Contagens (fora da ROI):
 - ROI Elipse χ² (p=95%) (normais) : 4037 (54.0%)
 - Q1 (Wr↑, Wm↑) : 3175 (42.5%)
 - Q2 (Wr↓, Wm↑) : 8 (0.1%)
 - Q3 (Wr↓, Wm↓) : 230 (3.1%)
 - Q4 (Wr↑, Wm↓) : 23 (0.3%)

Atribuição de drivers em Q1 (arquivo): C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\drivers_q1.csv
Q1_n                       3175.000000
peso_medio_v                  0.077052
peso_medio_|Delta|            0.905529
peso_medio_temp               0.000613
%Q1_onde_v_domina             1.763780
%Q1_onde_|Delta|_domina      98.236220
%Q1_onde_temp_domina 

In [1]:
# ============================================================
# v3 (REFERÊNCIA ROBUSTA): usar p5 (5º percentil) como Wr_ref/Wm_ref
# - Janela temporal: usa o "baseline anterior" (maior duração) se existir.
#   Caso a interseção dessa janela com o dataset v2 fique vazia, cai para "dataset inteiro".
# - Recalcula índices: Wr_idx_p5, Wm_idx_p5 (>= ~1 para ~95% dos pontos, salvo caudas).
# - Reporta: valores de referência, janela usada, e estatísticas de
#   vazão de AR (total ou primária) e vazão de CARVÃO (flw_total_c_t_h) no período.
# - Salva: tabela_wr_wm_com_desgaste_p5.parquet/.csv e um log JSON com o resumo.
# ============================================================
import os, json
import numpy as np
import pandas as pd

SRC_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL"
DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"

IN_V2   = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste_v2.parquet")
OUT_DIR = os.path.join(DST_BASE, r"data\derivadas")
LOG_DIR = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

OUT_PQ  = os.path.join(OUT_DIR, "tabela_wr_wm_com_desgaste_p5.parquet")
OUT_CSV = os.path.join(OUT_DIR, "tabela_wr_wm_com_desgaste_p5.csv")
OUT_LOG = os.path.join(LOG_DIR, "physics_refs_p5_summary.json")

# possíveis fontes do baseline "anterior"
CANDIDATES_BASELINE = [
    os.path.join(SRC_BASE, r"outputs\baseline_datasets\physics_baseline_proxies.csv"),
    os.path.join(SRC_BASE, r"outputs\baseline_mask.csv"),
]

# 1) Carregar v2 (Wr/Wm já com |Δ| e ε)
df = pd.read_parquet(IN_V2)
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.sort_values("timestamp")

# 2) Descobrir janela temporal do "baseline anterior" (maior duração)
baseline_range = None
baseline_source = None

# a) proxies.csv → usa min..max do timestamp
bp = CANDIDATES_BASELINE[0]
if os.path.exists(bp):
    try:
        bdf = pd.read_csv(bp, encoding="utf-8-sig")
    except Exception:
        bdf = pd.read_csv(bp, encoding="latin1")
    if "timestamp" in bdf.columns:
        bdf["timestamp"] = pd.to_datetime(bdf["timestamp"], errors="coerce")
        bdf = bdf.dropna(subset=["timestamp"])
        if not bdf.empty:
            baseline_range = (bdf["timestamp"].min(), bdf["timestamp"].max())
            baseline_source = os.path.relpath(bp, SRC_BASE)

# b) baseline_mask.csv → usa min..max das linhas com is_baseline==1
if baseline_range is None:
    bm = CANDIDATES_BASELINE[1]
    if os.path.exists(bm):
        try:
            mdf = pd.read_csv(bm, encoding="utf-8-sig")
        except Exception:
            mdf = pd.read_csv(bm, encoding="latin1")
        if {"timestamp","is_baseline"}.issubset(mdf.columns):
            mdf["timestamp"] = pd.to_datetime(mdf["timestamp"], errors="coerce")
            mdf = mdf[mdf["is_baseline"]==1].dropna(subset=["timestamp"])
            if not mdf.empty:
                baseline_range = (mdf["timestamp"].min(), mdf["timestamp"].max())
                baseline_source = os.path.relpath(bm, SRC_BASE)

# 3) Selecionar a janela para calcular p5
if baseline_range is not None:
    t0, t1 = baseline_range
    mask_win = (df["timestamp"] >= t0) & (df["timestamp"] <= t1)
    df_win = df.loc[mask_win].copy()
    # se a janela não intersecta o dataset v2, cair para tudo
    if df_win["Wr"].notna().sum() < 10 or df_win["Wm"].notna().sum() < 10:
        df_win = df.copy()
        baseline_source = "DATASET INTEIRO (fallback — baseline vazio)"
        t0, t1 = df_win["timestamp"].min(), df_win["timestamp"].max()
else:
    df_win = df.copy()
    t0, t1 = df_win["timestamp"].min(), df_win["timestamp"].max()
    baseline_source = "DATASET INTEIRO (sem baseline anterior)"

# 4) Referências robustas (p5) dentro da janela escolhida
Wr_ref_p5 = float(np.nanpercentile(pd.to_numeric(df_win["Wr"], errors="coerce"), 5))
Wm_ref_p5 = float(np.nanpercentile(pd.to_numeric(df_win["Wm"], errors="coerce"), 5))

# 5) Recalcular índices com p5
Wr_idx_p5 = df["Wr"] / Wr_ref_p5
Wm_idx_p5 = df["Wm"] / Wm_ref_p5

df_out = df.copy()
df_out["Wr_ref_p5"] = Wr_ref_p5
df_out["Wm_ref_p5"] = Wm_ref_p5
df_out["Wr_idx_p5"] = Wr_idx_p5
df_out["Wm_idx_p5"] = Wm_idx_p5

df_out.to_parquet(OUT_PQ, index=False)
df_out.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

# 6) Estatísticas operacionais na janela: vazão de AR e de CARVÃO (linha c)
# ar: preferir total; fallback = primário
air_col = "total_air_flow_knm3_h" if "total_air_flow_knm3_h" in df_win.columns else \
          ("total_paf_air_flow_knm3_h" if "total_paf_air_flow_knm3_h" in df_win.columns else None)
coal_col = "flw_total_c_t_h" if "flw_total_c_t_h" in df_win.columns else None

def stats_series(s):
    s = pd.to_numeric(s, errors="coerce")
    return {
        "count_valid": int(s.notna().sum()),
        "p5": float(np.nanpercentile(s, 5)) if s.notna().any() else None,
        "median": float(np.nanmedian(s)) if s.notna().any() else None,
        "p95": float(np.nanpercentile(s, 95)) if s.notna().any() else None,
        "mean": float(np.nanmean(s)) if s.notna().any() else None,
        "std": float(np.nanstd(s, ddof=1)) if s.notna().sum() > 1 else None,
        "unit_hint": "kNm³/h" if air_col and s.equals(pd.to_numeric(df_win[air_col], errors='coerce')) else ("t/h" if coal_col and s.equals(pd.to_numeric(df_win[coal_col], errors='coerce')) else None)
    }

air_stats = stats_series(df_win[air_col]) if air_col else None
coal_stats = stats_series(df_win[coal_col]) if coal_col else None

# 7) Log + prints
log = {
    "janela_usada": {"inicio": str(t0), "fim": str(t1), "fonte": baseline_source},
    "refs_p5": {"Wr_ref_p5": Wr_ref_p5, "Wm_ref_p5": Wm_ref_p5},
    "vazoes_no_periodo": {
        "ar_coluna": air_col,
        "ar_stats": air_stats,
        "carvao_coluna": coal_col,
        "carvao_stats": coal_stats
    },
    "arquivos_saida": {"parquet": OUT_PQ, "csv": OUT_CSV}
}
with open(OUT_LOG, "w", encoding="utf-8") as f:
    json.dump(log, f, ensure_ascii=False, indent=2)

print("[OK] v3 (p5) gerada:")
print(" -", OUT_PQ)
print(" -", OUT_CSV)
print("\n[REFERÊNCIAS (p5) NA JANELA]")
print(f"  Janela: {t0} → {t1}  |  Fonte: {baseline_source}")
print(f"  Wr_ref_p5 = {Wr_ref_p5}")
print(f"  Wm_ref_p5 = {Wm_ref_p5}")

if air_col:
    print(f"\n[Vazão de AR — coluna: {air_col}]")
    for k,v in air_stats.items():
        print(f"  {k}: {v}")
else:
    print("\n[Vazão de AR] coluna não encontrada.")

if coal_col:
    print(f"\n[Vazão de CARVÃO — coluna: {coal_col}]")
    for k,v in coal_stats.items():
        print(f"  {k}: {v}")
else:
    print("\n[Vazão de CARVÃO] coluna 'flw_total_c_t_h' não encontrada.")

print("\n[INFO] Log salvo em:", OUT_LOG)


[OK] v3 (p5) gerada:
 - C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_com_desgaste_p5.parquet
 - C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\data\derivadas\tabela_wr_wm_com_desgaste_p5.csv

[REFERÊNCIAS (p5) NA JANELA]
  Janela: 2024-03-04 20:00:00 → 2024-03-12 15:00:00  |  Fonte: outputs\baseline_datasets\physics_baseline_proxies.csv
  Wr_ref_p5 = 11218.414005750048
  Wm_ref_p5 = 64086720.957561284

[Vazão de AR — coluna: total_air_flow_knm3_h]
  count_valid: 188
  p5: 807.745
  median: 822.1505
  p95: 830.59695
  mean: 821.0314946808512
  std: 6.700817680567563
  unit_hint: kNm³/h

[Vazão de CARVÃO — coluna: flw_total_c_t_h]
  count_valid: 188
  p5: 61.6204
  median: 65.37450000000001
  p95: 71.19895
  mean: 65.86998936170214
  std: 2.9245689988036228
  unit_hint: t/h

[INFO] Log salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\physics_refs_p5_summary.json


In [3]:
# ============================================================
# Wr×Wm COM EIXOS NA REF P5 + ELIPSE χ²(95%)  (BRUTO e ÍNDICES)
# ============================================================
import os, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"

P5_PQ  = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste_p5.parquet")
P5_LOG = os.path.join(DST_BASE, r"outputs\diagnosticos\physics_refs_p5_summary.json")
OUT_DIR = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)

PNG_RAW = os.path.join(OUT_DIR, "wr_wm_elipse_ref_p5_raw.png")
PNG_IDX = os.path.join(OUT_DIR, "wr_wm_elipse_ref_p5_idx.png")
CSV_OUT_RAW = os.path.join(OUT_DIR, "wr_wm_outliers_elipse_raw.csv")
CSV_OUT_IDX = os.path.join(OUT_DIR, "wr_wm_outliers_elipse_idx.csv")

CHI2_P = 0.95
CHI2_TABLE = {0.90:4.605, 0.95:5.991, 0.99:9.210}
CHI2_THR = CHI2_TABLE[CHI2_P]

# --------- helpers ---------
def baseline_window_from_log(df, log_path):
    if not os.path.exists(log_path):
        return df["timestamp"].min(), df["timestamp"].max(), "DATASET INTEIRO (sem log p5)"
    with open(log_path, "r", encoding="utf-8") as f:
        meta = json.load(f)
    j = meta.get("janela_usada", {})
    t0 = pd.to_datetime(j.get("inicio"))
    t1 = pd.to_datetime(j.get("fim"))
    src = j.get("fonte", "desconhecida")
    if pd.isna(t0) or pd.isna(t1):
        return df["timestamp"].min(), df["timestamp"].max(), "DATASET INTEIRO (fallback)"
    return t0, t1, src

def cov_from_window(df_xy, t0, t1):
    win = df_xy[(df_xy["timestamp"]>=t0) & (df_xy["timestamp"]<=t1)]
    if len(win) < 10:
        win = df_xy  # fallback
    # Explicitly select only numeric columns for covariance calculation
    cov = np.cov(win["Wr"].values, win["Wm"].values)
    return cov

def ellipse_mask_and_patch(x, y, cx, cy, cov, chi2_thr, ax):
    # eigendecomp
    vals, vecs = np.linalg.eigh(cov)
    vals = np.clip(vals, 1e-12, None)
    inv_cov = np.linalg.inv(cov if np.all(np.isfinite(cov)) else np.diag(vals))
    # Mahalanobis
    diff = np.vstack([x - cx, y - cy]).T  # (n,2)
    d2 = np.einsum("ij,jk,ik->i", diff, inv_cov, diff)
    inside = d2 <= chi2_thr
    # ellipse patch (scale = sqrt(chi2))
    scale = np.sqrt(chi2_thr)
    width  = 2*scale*np.sqrt(vals[0])
    height = 2*scale*np.sqrt(vals[1])
    angle  = np.degrees(np.arctan2(vecs[1,0], vecs[0,0]))
    e = Ellipse((cx, cy), width, height, angle=angle, fill=False, linewidth=1.5)
    ax.add_patch(e)
    return inside

def quad_counts(x, y, cx, cy, outside_mask):
    q1 = outside_mask & (x>=cx) & (y>=cy)
    q2 = outside_mask & (x< cx) & (y>=cy)
    q3 = outside_mask & (x< cx) & (y< cy)
    q4 = outside_mask & (x>=cx) & (y< cy)
    return {
        "Q1 (↑,↑)": int(q1.sum()),
        "Q2 (↓,↑)": int(q2.sum()),
        "Q3 (↓,↓)": int(q3.sum()),
        "Q4 (↑,↓)": int(q4.sum()),
    }, q1, q2, q3, q4

def annotate_quads(ax, counts, cx, cy, xmin, xmax, ymin, ymax, total):
    xpadR = 0.18*(xmax - cx); xpadL = 0.30*(cx - xmin)
    ypadU = 0.18*(ymax - cy); ypadD = 0.30*(cy - ymin)
    box = dict(boxstyle="round,pad=0.2", fc="white", ec="gray", alpha=0.85)
    fmt = lambda n: f"{n} ({(100*n/total):.1f}%)"
    ax.text(cx + xpadR, cy + ypadU, f"Q1: {fmt(counts['Q1 (↑,↑)'])}", ha="left",  va="bottom", bbox=box)
    ax.text(cx - xpadL, cy + ypadU, f"Q2: {fmt(counts['Q2 (↓,↑)'])}", ha="right", va="bottom", bbox=box)
    ax.text(cx - xpadL, cy - ypadD, f"Q3: {fmt(counts['Q3 (↓,↓)'])}", ha="right", va="top",    bbox=box)
    ax.text(cx + xpadR, cy - ypadD, f"Q4: {fmt(counts['Q4 (↑,↓)'])}", ha="left",  va="top",    bbox=box)

# --------- 1) carregar p5 ---------
df = pd.read_parquet(P5_PQ)
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.sort_values("timestamp")

Wr_ref_p5 = float(df["Wr_ref_p5"].dropna().iloc[0])
Wm_ref_p5 = float(df["Wm_ref_p5"].dropna().iloc[0])

t0, t1, fonte = baseline_window_from_log(df, P5_LOG)

# ============================================================
# A) ESPAÇO BRUTO (Wr, Wm)
# ============================================================
raw = df[["timestamp","Wr","Wm"]].dropna().copy()
x, y = raw["Wr"].values, raw["Wm"].values
cx, cy = Wr_ref_p5, Wm_ref_p5

cov_raw = cov_from_window(raw, t0, t1)

fig, ax = plt.subplots(figsize=(8.4,8.4))
ax.scatter(x, y, s=10, alpha=0.35, label="Todos os pontos")
ax.axvline(cx, linestyle="--", linewidth=1)
ax.axhline(cy, linestyle="--", linewidth=1)

inside = ellipse_mask_and_patch(x, y, cx, cy, cov_raw, CHI2_THR, ax)
outside = ~inside
ax.scatter(x[inside],  y[inside],  s=10, alpha=0.65, label="Dentro elipse 95%")
ax.scatter(x[outside], y[outside], s=12, alpha=0.65, label="Fora elipse 95%")

counts, q1, q2, q3, q4 = quad_counts(x, y, cx, cy, outside)
xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()
annotate_quads(ax, counts, cx, cy, xmin, xmax, ymin, ymax, len(x))

ax.set_title(f"Wr × Wm (bruto) — eixos na ref p5, elipse χ²(95%)\nBaseline para Σ: {str(t0)} → {str(t1)} ({fonte})")
ax.set_xlabel("Wr (v2)")
ax.set_ylabel("Wm (v2)")
ax.legend(loc="upper left")
plt.tight_layout(); plt.savefig(PNG_RAW, dpi=150); plt.close()

# salvar CSV de outliers (bruto)
out_raw = raw.loc[outside, ["timestamp","Wr","Wm"]].copy()
out_raw["quadrante"] = np.where(q1, "Q1", np.where(q2, "Q2", np.where(q3, "Q3", "Q4")))[outside]
out_raw.to_csv(CSV_OUT_RAW, index=False, encoding="utf-8-sig")

print("[OK] RAW: gráfico ->", PNG_RAW)
print("     Fora elipse:", outside.sum(), "/", len(x), f"({100*outside.mean():.1f}%)")
for k,v in counts.items(): print("     ", k, ":", v)

# ============================================================
# B) ESPAÇO DE ÍNDICES (Wr_idx_p5, Wm_idx_p5)
# ============================================================
idx = df[["timestamp","Wr_idx_p5","Wm_idx_p5"]].dropna().copy()
xi, yi = idx["Wr_idx_p5"].values, idx["Wm_idx_p5"].values
cxi, cyi = 1.0, 1.0  # por definição da normalização p5

cov_idx = cov_from_window(idx.rename(columns={"Wr_idx_p5":"Wr","Wm_idx_p5":"Wm"}), t0, t1)

fig, ax = plt.subplots(figsize=(8.4,8.4))
ax.scatter(xi, yi, s=10, alpha=0.35, label="Todos os pontos (idx)")
ax.axvline(cxi, linestyle="--", linewidth=1)
ax.axhline(cyi, linestyle="--", linewidth=1)

inside_i = ellipse_mask_and_patch(xi, yi, cxi, cyi, cov_idx, CHI2_THR, ax)
outside_i = ~inside_i
ax.scatter(xi[inside_i],  yi[inside_i],  s=10, alpha=0.65, label="Dentro elipse 95%")
ax.scatter(xi[outside_i], yi[outside_i], s=12, alpha=0.65, label="Fora elipse 95%")

counts_i, q1i, q2i, q3i, q4i = quad_counts(xi, yi, cxi, cyi, outside_i)
xmin, xmax = xi.min(), xi.max()
ymin, ymax = yi.min(), yi.max()
annotate_quads(ax, counts_i, cxi, cyi, xmin, xmax, ymin, ymax, len(xi))

ax.set_title(f"Wr_idx_p5 × Wm_idx_p5 — eixos em (1,1), elipse χ²(95%)\nBaseline para Σ: {str(t0)} → {str(t1)} ({fonte})")
ax.set_xlabel("Wr_idx_p5")
ax.set_ylabel("Wm_idx_p5")
ax.legend(loc="upper left")
plt.tight_layout(); plt.savefig(PNG_IDX, dpi=150); plt.close()

# salvar CSV de outliers (índices)
out_idx = idx.loc[outside_i, ["timestamp","Wr_idx_p5","Wm_idx_p5"]].copy()
out_idx["quadrante"] = np.where(q1i, "Q1", np.where(q2i, "Q2", np.where(q3i, "Q3", "Q4")))[outside_i]
out_idx.to_csv(CSV_OUT_IDX, index=False, encoding="utf-8-sig")

print("[OK] IDX: gráfico ->", PNG_IDX)
print("     Fora elipse:", outside_i.sum(), "/", len(xi), f"({100*outside_i.mean():.1f}%)")
for k,v in counts_i.items(): print("     ", k, ":", v)


[OK] RAW: gráfico -> C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_wm_elipse_ref_p5_raw.png
     Fora elipse: 6729 / 7473 (90.0%)
      Q1 (↑,↑) : 6729
      Q2 (↓,↑) : 0
      Q3 (↓,↓) : 0
      Q4 (↑,↓) : 0
[OK] IDX: gráfico -> C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_wm_elipse_ref_p5_idx.png
     Fora elipse: 6729 / 7473 (90.0%)
      Q1 (↑,↑) : 6729
      Q2 (↓,↑) : 0
      Q3 (↓,↓) : 0
      Q4 (↑,↓) : 0


In [4]:
# ============================================================
# Wr×Wm com eixos nas referências p5 + elipse χ²(95%)
# E o mesmo gráfico no espaço dos ÍNDICES (Wr_idx_p5 × Wm_idx_p5)
# ------------------------------------------------------------
# FIX do erro DTypePromotionError:
#   - A covariância agora é calculada explicitamente nas colunas numéricas
#     (não usa iloc que pegava 'timestamp' sem querer).
# ============================================================
import os, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"

P5_PQ  = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste_p5.parquet")
P5_LOG = os.path.join(DST_BASE, r"outputs\diagnosticos\physics_refs_p5_summary.json")
OUT_DIR = os.path.join(DST_BASE, r"outputs\diagnosticos")
os.makedirs(OUT_DIR, exist_ok=True)

PNG_RAW = os.path.join(OUT_DIR, "wr_wm_elipse_ref_p5_raw.png")
PNG_IDX = os.path.join(OUT_DIR, "wr_wm_elipse_ref_p5_idx.png")
CSV_OUT_RAW = os.path.join(OUT_DIR, "wr_wm_outliers_elipse_raw.csv")
CSV_OUT_IDX = os.path.join(OUT_DIR, "wr_wm_outliers_elipse_idx.csv")

CHI2_P = 0.95
CHI2_TABLE = {0.90:4.605, 0.95:5.991, 0.99:9.210}
CHI2_THR = CHI2_TABLE[CHI2_P]

# ----------------- helpers -----------------
def baseline_window_from_log(df, log_path):
    if not os.path.exists(log_path):
        return df["timestamp"].min(), df["timestamp"].max(), "DATASET INTEIRO (sem log p5)"
    with open(log_path, "r", encoding="utf-8") as f:
        meta = json.load(f)
    j = meta.get("janela_usada", {})
    t0 = pd.to_datetime(j.get("inicio"))
    t1 = pd.to_datetime(j.get("fim"))
    src = j.get("fonte", "desconhecida")
    if pd.isna(t0) or pd.isna(t1):
        return df["timestamp"].min(), df["timestamp"].max(), "DATASET INTEIRO (fallback)"
    return t0, t1, src

def cov_from_window(df_xy, t0, t1, xcol, ycol):
    win = df_xy[(df_xy["timestamp"]>=t0) & (df_xy["timestamp"]<=t1)].copy()
    if len(win) < 10:
        win = df_xy.copy()  # fallback
    x = pd.to_numeric(win[xcol], errors="coerce").astype(float)
    y = pd.to_numeric(win[ycol], errors="coerce").astype(float)
    mask = x.notna() & y.notna()
    x = x[mask].values
    y = y[mask].values
    if x.size < 2:
        # variância ~0 -> usa diagonal pequena para não quebrar
        return np.diag([1e-12, 1e-12])
    cov = np.cov(x, y)
    # robustez numérica mínima
    if cov.shape != (2,2) or not np.all(np.isfinite(cov)):
        cov = np.diag([np.nanvar(x) + 1e-12, np.nanvar(y) + 1e-12])
    # ridge para garantir inversão
    cov = cov + np.eye(2)*1e-12
    return cov

def ellipse_mask_and_patch(x, y, cx, cy, cov, chi2_thr, ax):
    # eigendecomp para desenhar elipse
    vals, vecs = np.linalg.eigh(cov)
    vals = np.clip(vals, 1e-12, None)
    try:
        inv_cov = np.linalg.inv(cov)
    except np.linalg.LinAlgError:
        inv_cov = np.linalg.inv(np.diag(vals))
    # distância de Mahalanobis
    diff = np.vstack([x - cx, y - cy]).T  # (n,2)
    d2 = np.sum((diff @ inv_cov) * diff, axis=1)
    inside = d2 <= chi2_thr
    # patch da elipse (raios ~ sqrt(chi2)*sqrt(autovalor))
    scale = np.sqrt(chi2_thr)
    width  = 2*scale*np.sqrt(vals[0])
    height = 2*scale*np.sqrt(vals[1])
    angle  = np.degrees(np.arctan2(vecs[1,0], vecs[0,0]))
    e = Ellipse((cx, cy), width, height, angle=angle, fill=False, linewidth=1.5)
    ax.add_patch(e)
    return inside

def quad_counts(x, y, cx, cy, outside_mask):
    q1 = outside_mask & (x>=cx) & (y>=cy)
    q2 = outside_mask & (x< cx) & (y>=cy)
    q3 = outside_mask & (x< cx) & (y< cy)
    q4 = outside_mask & (x>=cx) & (y< cy)
    return {
        "Q1 (↑,↑)": int(q1.sum()),
        "Q2 (↓,↑)": int(q2.sum()),
        "Q3 (↓,↓)": int(q3.sum()),
        "Q4 (↑,↓)": int(q4.sum()),
    }, q1, q2, q3, q4

def annotate_quads(ax, counts, cx, cy, xmin, xmax, ymin, ymax, total):
    xpadR = 0.18*(xmax - cx); xpadL = 0.30*(cx - xmin)
    ypadU = 0.18*(ymax - cy); ypadD = 0.30*(cy - ymin)
    box = dict(boxstyle="round,pad=0.2", fc="white", ec="gray", alpha=0.85)
    fmt = lambda n: f"{n} ({(100*n/total):.1f}%)"
    ax.text(cx + xpadR, cy + ypadU, f"Q1: {fmt(counts['Q1 (↑,↑)'])}", ha="left",  va="bottom", bbox=box)
    ax.text(cx - xpadL, cy + ypadU, f"Q2: {fmt(counts['Q2 (↓,↑)'])}", ha="right", va="bottom", bbox=box)
    ax.text(cx - xpadL, cy - ypadD, f"Q3: {fmt(counts['Q3 (↓,↓)'])}", ha="right", va="top",    bbox=box)
    ax.text(cx + xpadR, cy - ypadD, f"Q4: {fmt(counts['Q4 (↑,↓)'])}", ha="left",  va="top",    bbox=box)

# ----------------- carregar p5 -----------------
df = pd.read_parquet(P5_PQ)
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.sort_values("timestamp")

Wr_ref_p5 = float(pd.to_numeric(df["Wr_ref_p5"], errors="coerce").dropna().iloc[0])
Wm_ref_p5 = float(pd.to_numeric(df["Wm_ref_p5"], errors="coerce").dropna().iloc[0])

t0, t1, fonte = baseline_window_from_log(df, P5_LOG)

# ============================================================
# A) ESPAÇO BRUTO (Wr, Wm) — eixos cruzando na ref p5
# ============================================================
raw = df[["timestamp","Wr","Wm"]].dropna().copy()
x = pd.to_numeric(raw["Wr"], errors="coerce").astype(float).values
y = pd.to_numeric(raw["Wm"], errors="coerce").astype(float).values
cx, cy = Wr_ref_p5, Wm_ref_p5

cov_raw = cov_from_window(raw, t0, t1, "Wr", "Wm")

fig, ax = plt.subplots(figsize=(8.4,8.4))
ax.scatter(x, y, s=10, alpha=0.35, label="Todos os pontos")
ax.axvline(cx, linestyle="--", linewidth=1)
ax.axhline(cy, linestyle="--", linewidth=1)

inside = ellipse_mask_and_patch(x, y, cx, cy, cov_raw, CHI2_THR, ax)
outside = ~inside
ax.scatter(x[inside],  y[inside],  s=10, alpha=0.65, label="Dentro elipse 95%")
ax.scatter(x[outside], y[outside], s=12, alpha=0.65, label="Fora elipse 95%")

counts, q1, q2, q3, q4 = quad_counts(x, y, cx, cy, outside)
xmin, xmax = np.nanmin(x), np.nanmax(x)
ymin, ymax = np.nanmin(y), np.nanmax(y)
annotate_quads(ax, counts, cx, cy, xmin, xmax, ymin, ymax, len(x))

ax.set_title(f"Wr × Wm (bruto) — eixos na ref p5, elipse χ²(95%)\nBaseline para Σ: {str(t0)} → {str(t1)} ({fonte})")
ax.set_xlabel("Wr (v2)")
ax.set_ylabel("Wm (v2)")
ax.legend(loc="upper left")
plt.tight_layout(); plt.savefig(PNG_RAW, dpi=150); plt.close()

# CSV outliers (bruto)
out_raw = raw.iloc[outside.nonzero()[0]][["timestamp","Wr","Wm"]].copy()
lab_raw = np.where(q1, "Q1", np.where(q2, "Q2", np.where(q3, "Q3", "Q4")))
out_raw["quadrante"] = lab_raw[outside]
out_raw.to_csv(CSV_OUT_RAW, index=False, encoding="utf-8-sig")

print("[OK] RAW: gráfico ->", PNG_RAW)
print("     Fora elipse:", int(outside.sum()), "/", len(x), f"({100*outside.mean():.1f}%)")
for k,v in counts.items(): print("     ", k, ":", v)

# ============================================================
# B) ESPAÇO DE ÍNDICES (Wr_idx_p5, Wm_idx_p5) — eixos (1,1)
# ============================================================
idx = df[["timestamp","Wr_idx_p5","Wm_idx_p5"]].dropna().copy()
xi = pd.to_numeric(idx["Wr_idx_p5"], errors="coerce").astype(float).values
yi = pd.to_numeric(idx["Wm_idx_p5"], errors="coerce").astype(float).values
cxi, cyi = 1.0, 1.0

cov_idx = cov_from_window(idx.rename(columns={"Wr_idx_p5":"Wr","Wm_idx_p5":"Wm"}),
                          t0, t1, "Wr", "Wm")

fig, ax = plt.subplots(figsize=(8.4,8.4))
ax.scatter(xi, yi, s=10, alpha=0.35, label="Todos os pontos (idx)")
ax.axvline(cxi, linestyle="--", linewidth=1)
ax.axhline(cyi, linestyle="--", linewidth=1)

inside_i = ellipse_mask_and_patch(xi, yi, cxi, cyi, cov_idx, CHI2_THR, ax)
outside_i = ~inside_i
ax.scatter(xi[inside_i],  yi[inside_i],  s=10, alpha=0.65, label="Dentro elipse 95%")
ax.scatter(xi[outside_i], yi[outside_i], s=12, alpha=0.65, label="Fora elipse 95%")

counts_i, q1i, q2i, q3i, q4i = quad_counts(xi, yi, cxi, cyi, outside_i)
xmin, xmax = np.nanmin(xi), np.nanmax(xi)
ymin, ymax = np.nanmin(yi), np.nanmax(yi)
annotate_quads(ax, counts_i, cxi, cyi, xmin, xmax, ymin, ymax, len(xi))

ax.set_title(f"Wr_idx_p5 × Wm_idx_p5 — eixos em (1,1), elipse χ²(95%)\nBaseline para Σ: {str(t0)} → {str(t1)} ({fonte})")
ax.set_xlabel("Wr_idx_p5")
ax.set_ylabel("Wm_idx_p5")
ax.legend(loc="upper left")
plt.tight_layout(); plt.savefig(PNG_IDX, dpi=150); plt.close()

# CSV outliers (índices)
out_idx = idx.iloc[outside_i.nonzero()[0]][["timestamp","Wr_idx_p5","Wm_idx_p5"]].copy()
lab_idx = np.where(q1i, "Q1", np.where(q2i, "Q2", np.where(q3i, "Q3", "Q4")))
out_idx["quadrante"] = lab_idx[outside_i]
out_idx.to_csv(CSV_OUT_IDX, index=False, encoding="utf-8-sig")

print("[OK] IDX: gráfico ->", PNG_IDX)
print("     Fora elipse:", int(outside_i.sum()), "/", len(xi), f"({100*outside_i.mean():.1f}%)")
for k,v in counts_i.items(): print("     ", k, ":", v)


[OK] RAW: gráfico -> C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_wm_elipse_ref_p5_raw.png
     Fora elipse: 6729 / 7473 (90.0%)
      Q1 (↑,↑) : 6729
      Q2 (↓,↑) : 0
      Q3 (↓,↓) : 0
      Q4 (↑,↓) : 0
[OK] IDX: gráfico -> C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\diagnosticos\wr_wm_elipse_ref_p5_idx.png
     Fora elipse: 6729 / 7473 (90.0%)
      Q1 (↑,↑) : 6729
      Q2 (↓,↑) : 0
      Q3 (↓,↓) : 0
      Q4 (↑,↓) : 0


In [5]:
# ================== CHECK: contagens e diagnóstico ==================
import os, json
import numpy as np
import pandas as pd
from numpy.linalg import inv, eigh

DST_BASE = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO"
P5_PQ  = os.path.join(DST_BASE, r"data\derivadas\tabela_wr_wm_com_desgaste_p5.parquet")
P5_LOG = os.path.join(DST_BASE, r"outputs\diagnosticos\physics_refs_p5_summary.json")

CHI2_THR = 5.991  # 95%, 2 g.l.

df = pd.read_parquet(P5_PQ).sort_values("timestamp")
# --- totais vs. o que entra no gráfico de índices ---
n_total = len(df)
idx = df[["timestamp","Wr_idx_p5","Wm_idx_p5"]].dropna().copy()
n_plot = len(idx)
print(f"[TOTAL] linhas na base: {n_total:,}")
print(f"[PLOT ] linhas com Wr_idx_p5 & Wm_idx_p5 válidos: {n_plot:,} ({n_plot/n_total:.1%})")

# --- diagnóstico do porquê ficaram de fora (usa causas na base v2) ---
miss = df[(df["Wr"].isna()) | (df["Wm"].isna())].copy()
v_missing = (df.get("total_air_flow_knm3_h").isna() if "total_air_flow_knm3_h" in df.columns else True) & \
            (df.get("total_paf_air_flow_knm3_h").isna() if "total_paf_air_flow_knm3_h" in df.columns else True)
delta_missing = df.get("delta_proxy").isna() if "delta_proxy" in df.columns else True
tau_missing = pd.concat([df[c] for c in ["tau_densa","tau_diluida","tau_backpass"] if c in df.columns], axis=1).isna().all(axis=1)

print("\n[CAUSAS para linhas fora do gráfico (Wr/Wm NaN) — contagens]")
print(" - faltou vazão de AR (total e primária ao mesmo tempo):", int(v_missing.sum()))
print(" - faltou delta_proxy:", int(delta_missing.sum()))
print(" - faltaram TODAS as taus (densa/diluida/backpass):", int(tau_missing.sum()))

# --- janela do baseline para a elipse ---
if os.path.exists(P5_LOG):
    with open(P5_LOG, "r", encoding="utf-8") as f:
        meta = json.load(f)
    j = meta.get("janela_usada", {})
    t0 = pd.to_datetime(j.get("inicio")); t1 = pd.to_datetime(j.get("fim"))
else:
    t0 = idx["timestamp"].min(); t1 = idx["timestamp"].max()

# --- elipse 95% no espaço dos ÍNDICES (centrada em 1,1) ---
win = idx[(idx["timestamp"]>=t0) & (idx["timestamp"]<=t1)]
if len(win) < 10: win = idx
X = win["Wr_idx_p5"].astype(float).values
Y = win["Wm_idx_p5"].astype(float).values
cov = np.cov(X, Y) + np.eye(2)*1e-12

# Mahalanobis de TODOS os pontos de índice
xi = idx["Wr_idx_p5"].astype(float).values
yi = idx["Wm_idx_p5"].astype(float).values
diff = np.vstack([xi - 1.0, yi - 1.0]).T
inv_cov = inv(cov)
d2 = np.sum((diff @ inv_cov) * diff, axis=1)
inside = d2 <= CHI2_THR
n_inside = int(inside.sum()); n_out = int((~inside).sum())

print(f"\n[ELIPSE 95% nos ÍNDICES]")
print(f" - janela para Σ: {t0} → {t1}")
print(f" - dentro da elipse: {n_inside:,} ({n_inside/n_plot:.1%} dos {n_plot:,} plotados)")
print(f" - fora   da elipse: {n_out:,} ({n_out/n_plot:.1%} dos {n_plot:,} plotados)")


[TOTAL] linhas na base: 11,757
[PLOT ] linhas com Wr_idx_p5 & Wm_idx_p5 válidos: 7,473 (63.6%)

[CAUSAS para linhas fora do gráfico (Wr/Wm NaN) — contagens]
 - faltou vazão de AR (total e primária ao mesmo tempo): 10
 - faltou delta_proxy: 0
 - faltaram TODAS as taus (densa/diluida/backpass): 0

[ELIPSE 95% nos ÍNDICES]
 - janela para Σ: 2024-03-04 20:00:00 → 2024-03-12 15:00:00
 - dentro da elipse: 744 (10.0% dos 7,473 plotados)
 - fora   da elipse: 6,729 (90.0% dos 7,473 plotados)


In [1]:
# ============================================================
# PROJETO A1 — Empacote do MODELO FÍSICO (igual ao E0)
# ============================================================
import os, re, json, math, joblib, numpy as np, pandas as pd
from datetime import datetime

# --------- PATHS (ajuste se necessário)
FREEZE_DIR = r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\freeze\v1_20250818_0635"
PATH_ROT_GOLD = os.path.join(FREEZE_DIR, "A1_ML_DL_rotulado_v4_gold.csv")   # insumos físicos costumam estar aqui
PHYS_DIR      = os.path.join(FREEZE_DIR, "models", "phys")
MVP_DIR       = os.path.join(FREEZE_DIR, "models", "mvp")                   # onde já fixamos o E0

os.makedirs(PHYS_DIR, exist_ok=True)
os.makedirs(MVP_DIR,  exist_ok=True)

# ============================================================
# 1) Implementação do modelo físico (MVP)
# ============================================================
class PhysicsBasedModel:
    """
    Modelo físico determinístico para Wr/Wm.
    Fórmulas (MVP):

      Wr = α · ν^2 · δ^1 · exp(−T/Tcrit) · g
      Wm = β · ν^3 · √δ · exp(−T/Topt) · h

      com g = σ_mat / ρ_fuel
          h = σ_steel / μ_gas

    Notas:
      - Unidades esperadas (convenção MVP):
        ν [m/s], δ [m], T [K], μ_gas [Pa·s], ρ_fuel [kg/m³], σ_mat e σ_steel (adimensionais/escalares).
      - Os coeficientes (α, β, Tcrit, Topt, C_DELTA) são os "knobs" do MVP.
      - Se insumos alternativos estiverem presentes (ex.: QN_total, T_gás, P), pode-se
        preparar ν, μ_gas, etc., antes de chamar predict (ou criar um pré-processador).
    """

    def __init__(self,
                 alpha=1.0, beta=1.0,
                 Tcrit=1400.0, Topt=1100.0,
                 ):
        self.alpha = float(alpha)
        self.beta  = float(beta)
        self.Tcrit = float(Tcrit)
        self.Topt  = float(Topt)

        # Campos (nomes canônicos) esperados no DataFrame de entrada:
        # você pode alterar/expandir esse mapeamento se seus nomes forem diferentes.
        self.required = [
            "nu_m_s",             # ν (velocidade)
            "delta_eff_m",        # δ (diâmetro/escala efetiva)
            "T_gas_K",            # T (Kelvin)
            "mu_gas_Pa_s",        # μ_gas
            "sigma_mat",          # σ_mat
            "rho_fuel_kg_m3",     # ρ_fuel
            "sigma_steel",        # σ_steel
        ]

    # --- utilitários (opcionais) ---
    @staticmethod
    def sutherland_mu(TK, mu0=1.716e-5, T0=273.15, S=110.4):
        """Viscosidade do ar (Pa·s) pela lei de Sutherland (aproximação)."""
        TK = np.clip(np.asarray(TK, dtype=float), 1.0, None)
        return mu0 * (T0 + S)/(TK + S) * (TK/T0)**1.5

    def predict(self, df: pd.DataFrame) -> pd.DataFrame:
        # Verifica insumos
        missing = [c for c in self.required if c not in df.columns]
        if missing:
            raise ValueError(
                "Faltam insumos para o modelo físico: "
                + ", ".join(missing)
                + ".\nAdapte os nomes/prepare os sinais antes de chamar .predict()."
            )

        # Extrai vetores
        nu   = pd.to_numeric(df["nu_m_s"], errors="coerce").values          # m/s
        delt = pd.to_numeric(df["delta_eff_m"], errors="coerce").values     # m
        TK   = pd.to_numeric(df["T_gas_K"], errors="coerce").values         # K
        mu   = pd.to_numeric(df["mu_gas_Pa_s"], errors="coerce").values     # Pa·s
        smat = pd.to_numeric(df["sigma_mat"], errors="coerce").values
        rhoF = pd.to_numeric(df["rho_fuel_kg_m3"], errors="coerce").values  # kg/m³
        sstl = pd.to_numeric(df["sigma_steel"], errors="coerce").values

        # Fatores g e h
        # Proteções para zero/NaN
        rhoF = np.where(np.abs(rhoF) < 1e-12, np.nan, rhoF)
        mu   = np.where(np.abs(mu)   < 1e-12, np.nan, mu)

        g = smat / rhoF
        h = sstl / mu

        # Componentes
        term_wr = (nu**2) * (delt**1.0) * np.exp(-TK / self.Tcrit) * g
        term_wm = (nu**3) * np.sqrt(np.clip(delt, 0.0, None)) * np.exp(-TK / self.Topt) * h

        # Saídas
        Wr = self.alpha * term_wr
        Wm = self.beta  * term_wm

        out = pd.DataFrame({
            "wr_kg_m2_h_pred_phys": Wr,
            "wm_kg_m2_h_pred_phys": Wm,
        })
        return out

# ============================================================
# 2) (Opcional) Auto-mapeamento de colunas no rotulado GOLD
#     -> tenta encontrar sinais físicos com heurísticas de nome
# ============================================================
def _auto_map_columns(df: pd.DataFrame):
    """
    Heurística simples para mapear colunas do rotulado para os canônicos:
      - nu_m_s:         r'(?:^|_)(nu|velocidade|velocity).*'
      - delta_eff_m:    r'(?:^|_)(delta|diam(etro)?|d_eff).*'
      - T_gas_K:        r'(?:^|_)(t_gas|temp.*gas|tk|t_k|t_gas_k).*'
      - mu_gas_Pa_s:    r'(?:^|_)(mu_gas|viscos.*gas).*'
      - sigma_mat:      r'(?:^|_)(sigma_mat|sig_mat|s_mat).*'
      - rho_fuel_kg_m3: r'(?:^|_)(rho_fuel|dens.*comb|dens.*fuel).*'
      - sigma_steel:    r'(?:^|_)(sigma_steel|sig_steel|s_steel).*'
    """
    patterns = {
        "nu_m_s":          re.compile(r'(?:^|_)(nu|velocidade|velocity)', re.I),
        "delta_eff_m":     re.compile(r'(?:^|_)(delta|diam(?:etro)?|d_eff)', re.I),
        "T_gas_K":         re.compile(r'(?:^|_)(t_gas|temp.*gas|tk\b|t_k\b|t_gas_k)', re.I),
        "mu_gas_Pa_s":     re.compile(r'(?:^|_)(mu_gas|viscos.*gas)', re.I),
        "sigma_mat":       re.compile(r'(?:^|_)(sigma_mat|sig_mat|s_mat)', re.I),
        "rho_fuel_kg_m3":  re.compile(r'(?:^|_)(rho_fuel|dens.*comb|dens.*fuel)', re.I),
        "sigma_steel":     re.compile(r'(?:^|_)(sigma_steel|sig_steel|s_steel)', re.I),
    }
    colmap = {}
    cols = list(df.columns)
    for target, pat in patterns.items():
        for c in cols:
            if pat.search(str(c)):
                colmap[target] = c
                break
    return colmap

# ============================================================
# 3) Carrega rotulado GOLD (se existir) e tenta verificação
# ============================================================
df_rot = None
colmap  = {}
verified = False
verification_stats = None
sample_preds_path = None

if os.path.exists(PATH_ROT_GOLD):
    dfr = pd.read_csv(PATH_ROT_GOLD, header=[0,1], engine="python")
    df_rot = dfr.copy()
    df_rot.columns = [c for (c,_) in dfr.columns]  # usa linha 1 (nomes)
    # tenta mapear
    colmap = _auto_map_columns(df_rot)
    # renomeia cópia se houver mapeamento mínimo
    needed = ["nu_m_s","delta_eff_m","T_gas_K","mu_gas_Pa_s","sigma_mat","rho_fuel_kg_m3","sigma_steel"]
    has_min = all(k in colmap for k in needed if k not in ["sigma_mat","rho_fuel_kg_m3","sigma_steel"])  # tolera faltas nos escalares
else:
    has_min = False

# ============================================================
# 4) Instancia o modelo e (se der) verifica em amostra
# ============================================================
phys = PhysicsBasedModel(
    alpha=1.0,   # deixe 1.0 — coeficientes do MVP podem ser ajustados depois se desejado
    beta=1.0,
    Tcrit=1400.0,
    Topt=1100.0
)

if df_rot is not None and has_min:
    # monta DF de entrada para o modelo físico
    df_in = pd.DataFrame(index=df_rot.index)

    def pick(name, default=None):
        if name in colmap:
            return pd.to_numeric(df_rot[colmap[name]], errors="coerce")
        else:
            return pd.Series(default, index=df_rot.index, dtype="float64")

    df_in["nu_m_s"]          = pick("nu_m_s", np.nan)
    df_in["delta_eff_m"]     = pick("delta_eff_m", np.nan)
    df_in["T_gas_K"]         = pick("T_gas_K", np.nan)
    df_in["mu_gas_Pa_s"]     = pick("mu_gas_Pa_s", phys.sutherland_mu(df_rot[colmap.get("T_gas_K", df_rot.columns[0])]) if "T_gas_K" in colmap else np.nan)
    df_in["sigma_mat"]       = pick("sigma_mat", 1.0)         # default neutro
    df_in["rho_fuel_kg_m3"]  = pick("rho_fuel_kg_m3", 1.0)    # evita div/0
    df_in["sigma_steel"]     = pick("sigma_steel", 1.0)

    # predição física
    preds = phys.predict(df_in)

    # se rótulos estiverem no rotulado, podemos checar (Wr/Wm)
    ycols = []
    for y in ["wr_kg_m2_h","wm_kg_m2_h"]:
        if y in df_rot.columns:
            ycols.append(y)
    if len(ycols) == 2:
        err_wr = (pd.to_numeric(df_rot["wr_kg_m2_h"], errors="coerce") - preds["wr_kg_m2_h_pred_phys"]).abs().median()
        err_wm = (pd.to_numeric(df_rot["wm_kg_m2_h"], errors="coerce") - preds["wm_kg_m2_h_pred_phys"]).abs().median()
        verification_stats = {"median_abs_err_wr": float(err_wr), "median_abs_err_wm": float(err_wm)}
        verified = True

    # salva uma amostra pequena
    sample = pd.concat([df_in.head(200), preds.head(200)], axis=1)
    sample_preds_path = os.path.join(PHYS_DIR, "sample_preds_phys.csv")
    sample.to_csv(sample_preds_path, index=False, encoding="utf-8")

# ============================================================
# 5) Persistência: joblib + manifesto + card
# ============================================================
tag = datetime.now().strftime("%Y%m%d_%H%M")
phys_path = os.path.join(PHYS_DIR, "model_phys_mvp.joblib")
joblib.dump({
    "model": phys,
    "y_names": ["wr_kg_m2_h","wm_kg_m2_h"],
    "input_names_canonical": phys.required,
    "notes": "Deterministic physics-based MVP. Provide inputs with canonical names/units.",
}, phys_path)

manifest = {
    "tag": "MVP_PHYSICS",
    "created_at": datetime.now().isoformat(),
    "freeze_dir": FREEZE_DIR,
    "artifact": os.path.basename(phys_path),
    "y_names": ["wr_kg_m2_h","wm_kg_m2_h"],
    "inputs_expected": phys.required,
    "defaults": {"sigma_mat": 1.0, "rho_fuel_kg_m3": 1.0, "sigma_steel": 1.0},
    "formulae": {
        "Wr": "alpha * (nu^2) * (delta^1) * exp(-T/Tcrit) * (sigma_mat / rho_fuel)",
        "Wm": "beta  * (nu^3) * sqrt(delta) * exp(-T/Topt) * (sigma_steel / mu_gas)"
    },
    "constants": {"alpha": phys.alpha, "beta": phys.beta, "Tcrit": phys.Tcrit, "Topt": phys.Topt},
    "auto_mapping_used": colmap if colmap else {},
    "verification": {
        "performed": bool(verified),
        "stats": verification_stats,
        "sample_preds": sample_preds_path if sample_preds_path else None
    }
}
manifest_path = os.path.join(PHYS_DIR, "manifest_phys_mvp.json")
with open(manifest_path, "w", encoding="utf-8") as f:
    json.dump(manifest, f, indent=2, ensure_ascii=False)

# Model card (curto)
card = f"""
# Projeto A1 — Model Card (Physics-Based MVP)

## 1) Identificação
- **Modelo:** PhysicsBasedModel (determinístico)
- **Tag:** MVP_PHYSICS
- **Freeze dir:** {FREEZE_DIR}
- **Arquivo:** models\\phys\\model_phys_mvp.joblib
- **Gerado em:** {datetime.now().date().isoformat()}

## 2) Fórmulas
- Wr = α · ν² · δ¹ · exp(−T/Tcrit) · (σ_mat/ρ_fuel)
- Wm = β · ν³ · √δ · exp(−T/Topt) · (σ_steel/μ_gas)

## 3) Insumos esperados (nomes canônicos)
{', '.join(phys.required)}

## 4) Constantes (MVP)
α={phys.alpha}, β={phys.beta}, Tcrit={phys.Tcrit} K, Topt={phys.Topt} K

## 5) Notas
- Unidade esperada: ν[m/s], δ[m], T[K], μ_gas[Pa·s], ρ_fuel[kg/m³].
- O artefato **não aprende**; aplica equações físicas. Calibração de α/β/Tcrit/Topt pode ser feita em futura versão.
- Se o CSV tiver nomes diferentes, renomeie ou mapeie para os canônicos antes de inferir.

## 6) Verificação
- Executada: {"Sim" if verified else "Não"}
- Estatísticas: {json.dumps(verification_stats) if verification_stats else "—"}
- Amostra de predição: {sample_preds_path if sample_preds_path else "—"}
"""
card_path = os.path.join(PHYS_DIR, "model_card_phys.md")
with open(card_path, "w", encoding="utf-8") as f:
    f.write(card.strip()+"\n")

# ============================================================
# 6) Relatório final de caminhos (E0 e Físico)
# ============================================================
e0_model_path   = os.path.join(MVP_DIR, "model_mvp.joblib")
e0_manifest     = os.path.join(MVP_DIR, "manifest_mvp.json")
e0_card         = os.path.join(MVP_DIR, "model_card.md")

print("======== ARTEFATOS SALVOS ========")
print("E0 (MVP supervisionado):")
print("  - Modelo:   ", e0_model_path)
print("  - Manifesto:", e0_manifest)
print("  - Card:     ", e0_card)
print("")
print("Modelo Físico (MVP):")
print("  - Modelo:   ", phys_path)
print("  - Manifesto:", manifest_path)
print("  - Card:     ", card_path)
if sample_preds_path:
    print("  - Amostra preds:", sample_preds_path)
print("==================================")


======== ARTEFATOS SALVOS ========
E0 (MVP supervisionado):
  - Modelo:    C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\freeze\v1_20250818_0635\models\mvp\model_mvp.joblib
  - Manifesto: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\freeze\v1_20250818_0635\models\mvp\manifest_mvp.json
  - Card:      C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\freeze\v1_20250818_0635\models\mvp\model_card.md

Modelo Físico (MVP):
  - Modelo:    C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\freeze\v1_20250818_0635\models\phys\model_phys_mvp.joblib
  - Manifesto: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\freeze\v1_20250818_0635\models\phys\manifest_phys_mvp.json
  - Card:      C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL_REFAZIMENTO\outputs\freeze\v1_20250818_0635\models\phys\model_card_phys.md
